In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  6 17:42:33 2020

@author: Liang
"""
import pandas as pd
import os
import numpy as np
import time
import datetime


#输出文件：XXX.csv
input_file

#检查1or2输入
def safe_1or2_input(prompt):
    inputString = input(prompt)
    try:
        number = int(inputString)
        if number == 1:
            print('您输入的是1。')
            return number
        elif number == 2:
            print('您输入的是2。')
            return number
        else:
            print('错误的输入：', inputString)
            print('请输入1或者2')
            return safe_1or2_input(prompt)
    except ValueError:
        print('错误的输入：', inputString)
        print('请输入1或者2')
        return safe_1or2_input(prompt)

#清洗：交易卡号、交易账号去除 “_” 下划线后的值
def delete_dash(df2):
    df2['交易卡号'] = df2['交易卡号'].str.split('_',expand = True)[0]
    df2['交易账号'] = df2['交易账号'].str.split('_',expand = True)[0]
    return df2

#清洗：交易卡号为空就填充交易账号的值，交易账号为空就填充交易卡号的值
def fill_value(df2):
    df2[['交易卡号','交易账号']] = df2[['交易卡号','交易账号']].fillna(axis=1,method='backfill')
    df2[['交易卡号','交易账号']] = df2[['交易卡号','交易账号']].fillna(axis=1,method='ffill')
    return df2

#开户信息去除重复记录
def remove_duplicate_kaihu(df2):
    #按照"交易户名"、"交易卡号"、"交易账号"升序排序
    df2 = df2.sort_values(by=['交易户名','交易卡号','交易账号'])
    #依据‘交易卡号’、‘交易账号’删除重复记录，保留第一条（有户名的）
    df2.drop_duplicates(['交易卡号','交易账号'],'first',inplace=True)
    df2.reset_index(drop= True,inplace=True)
    return df2

#清洗：依据开户信息表，填充相应空值，表拼接（字段同名）
def splicing_table(df0,df2,null_name,link_name):
    #筛选出null_name为空的行
    df3 = df2[df2[null_name].isna()]
    if df3.empty is False:
        #依据link_name对应将两个表连接，并填充null_name的值
        df3 = pd.merge(df3,df0.loc[:,[link_name,null_name]],\
                       how='left',left_on=link_name, right_on=link_name)
        df3[null_name+'_x'] = df3[null_name+'_y']
        #删除多余列,修改列名
        df3.drop([null_name+'_y'], axis=1,inplace=True)
        df3.rename(columns={null_name+'_x':null_name},inplace=True)
        #去重
        df2 = df2.append(df3,sort=False)
        df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    return df2

#清洗：null_name为空根据已有的null_name-link_name对应进行填充（字段同名）
def splicing_alreadytable(df2,null_name,link_name):
    #明细中已有的null_name-link_name对应关系
    df4 = df2[[null_name,link_name]].copy()
    #删除包含空值的行，并去重
    df4.dropna(how = 'any',axis = 0,inplace=True)
    df4.drop_duplicates(subset=None,keep='first',inplace=True)
    #筛选出null_name为空的行
    df3 = df2[df2[null_name].isna()]
    if df3.empty is False:
        #依据已有的link_name对应关系连接
        df3= pd.merge(df3,df4,how='left',left_on=link_name, right_on=link_name)
        df3[null_name+'_x'] = df3[null_name+'_y']
        #删除多余列,修改列名
        df3.drop([null_name+'_y'], axis=1,inplace=True)
        df3.rename(columns={null_name+'_x':null_name},inplace=True)
        #去重
        df2 = df2.append(df3,sort=False)
        df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    return df2

#清洗：null_name为空根据已有的null_nameB-link_nameB对应进行填充（字段不同名）
def splicing_alreadytableB(df2,null_nameA,link_nameA,null_nameB,link_nameB):
    #明细中已有的null_nameB-link_nameB对应关系
    df4 = df2[[null_nameB,link_nameB]].copy()
    #删除包含空值的行，并去重
    df4.dropna(how = 'any',axis = 0,inplace=True)
    df4.drop_duplicates(subset=None,keep='first',inplace=True)
    #筛选出null_nameA为空的行
    df3 = df2[df2[null_nameA].isna()]
    if df3.empty is False:
        #依据已有的link_name对应关系连接
        df3= pd.merge(df3,df4,how='left',left_on=link_nameA, right_on=link_nameB)
        df3[null_nameA] = df3[null_nameB+'_y']
        #删除多余列,修改列名
        df3.drop([null_nameB+'_y',link_nameB+'_y'], axis=1,inplace=True)
        df3.rename(columns={null_nameB+'_x':null_nameB,link_nameB+'_x':link_nameB},inplace=True)
        #去重
        df2 = df2.append(df3,sort=False)
        df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    return df2

#清洗：删除对手卡号、对手账号等于“0”
def remove_cardB_0(df2):
    card_dict = {'0':np.nan,\
                 '00':np.nan,\
                     '000':np.nan,\
                         '00000000':np.nan,\
                             '000000000000000':np.nan}
    df2.loc[:,['对手卡号','对手账号']].replace(card_dict,inplace=True)
    return df2

#找出卡号-账号的对应关系（不看卡号账号一样的），全局替换，将账号替换成卡号
def replace_card(df2,value,key):
    df4 = df2[[value,key]].copy()
    #删除包含空值的行，并去重
    df4.dropna(how = 'any',axis = 0,inplace=True)
    df4.drop_duplicates(subset=None,keep='first',inplace=True)
    df4.reset_index(drop= True,inplace=True)
        
    df4 = df4[df4[value] != df4[key]]
    
    if df4.empty is False:
        card_dict = df4.set_index(key)[value].to_dict()
        df2.loc[:,['交易卡号','对手卡号']].replace(card_dict,inplace=True)
    return df2

#清洗：交易金额为负，借贷标志为出，则将金额取正，借贷标志设为进（冲正、消费退货、当日冲账）
def reversal(df2):
    df3 = df2[(df2['交易金额']<0) & (df2['借贷方向']== '出')].copy()
    if df3.empty is False:
        df3.loc[:,'交易金额'] = df3.loc[:,'交易金额'].abs()
        df3.loc[:,'借贷方向'] = '进'   
        #去重
        df2 = df2.append(df3,sort=False)
        df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    return df2

#清洗：交易金额为负，借贷标志为进，则将金额取正，借贷标志设为出
def reversal1(df2):
    df3 = df2[(df2['交易金额']<0) & (df2['借贷方向']=='进')].copy()
    if df3.empty is False:
        df3.loc[:,'交易金额'] = df3.loc[:,'交易金额'].abs()
        df3.loc[:,'借贷方向'] = '出'
        #去重
        df2 = df2.append(df3,sort=False)
        df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    return df2

#清洗：删除摘要包含“结息”“利息”记录
def delete_interest(df2):
    df3 = df2[df2['摘要'].notnull()]
    df3 = df3[~ df3['摘要'].str.contains('结息|利息')]
    return df3

#清洗：删除支付宝、财付通、京东账号
def delete_public_number(df2):
    third_party = {'105450160110003':np.nan,\
                    '105340160110001':np.nan,\
                    '105430160110002':np.nan,\
                    '105220160110002':np.nan,\
                    '105130160110008':np.nan,\
                    '105330260110001':np.nan,\
                    '105450173990085':np.nan,\
                    '105331000000875':np.nan,\
                    '105440173990076':np.nan,\
                    '105440160110002':np.nan,\
                    '105330273990212':np.nan,\
                    '105350273990003':np.nan,\
                    '105350160110009':np.nan,\
                    '105360160110001':np.nan,\
                    '105340173990228':np.nan,\
                    '105350173990040':np.nan,\
                    '105584000000016':np.nan,\
                    '105360173990025':np.nan,\
                    '105350260110002':np.nan,\
                    '815110554110031':np.nan,\
                    '790660120000601':np.nan,\
                    '1500947831':np.nan,\
                    '815110554110051':np.nan,\
                    '105584073990267':np.nan,\
                    '105100000002625':np.nan,\
                    '818331060120004':np.nan,\
                    '105290073990576':np.nan,\
                    '812310060120099':np.nan,\
                    '105110073991392':np.nan,\
                    '105100000018654':np.nan,\
                    '812310060120038':np.nan,\
                    '812310060120098':np.nan,\
                    '825922079930001':np.nan,\
                    '812440360120003':np.nan,\
                    '833012457350003':np.nan,\
                    '105330173990081':np.nan,\
                    '105330160110001':np.nan,\
                    '105510173990804':np.nan,\
                    '105110089991074':np.nan,\
                    '888331073999999':np.nan,\
                    '027430148149999':np.nan,\
                    '105100000022763':np.nan,\
                    '105584072990020':np.nan,\
                    '102440183986056':np.nan,\
                    '105331000003156':np.nan,\
                    '105110073991437':np.nan,\
                    '105584000000061':np.nan,\
                    '105320160110001':np.nan,\
                    '99010133190000165':np.nan,\
                    '79010133190000012':np.nan,\
                    '825922048160001':np.nan,\
                    '001980099990002':np.nan,\
                    '105110089999093':np.nan,\
                    '105320173990018':np.nan,\
                    '105110073992138':np.nan,\
                    '105551000020704':np.nan,\
                    '95010133190000012':np.nan,\
                    '818922060129999':np.nan,\
                    '105584073990213':np.nan,\
                    '105410160110003':np.nan,\
                    '105330173990049':np.nan,\
                    '493611004816001':np.nan,\
                    '105430173990408':np.nan,\
                    '105290000030844':np.nan,\
                    '215500690':np.nan,\
                    '243300133':np.nan,\
                    '33001616783059000667':np.nan,\
                    '401110048164078':np.nan,\
                    '03317300040010748':np.nan,\
                    '310113148164032':np.nan,\
                    '991290000015':np.nan,\
                    '309113148164020':np.nan,\
                    '317113148163116':np.nan,\
                    '41007800040010999':np.nan,\
                    '401110048164018':np.nan,\
                    '99999999999999999':np.nan,\
                    '410350248160069':np.nan,\
                    '41007801941001021':np.nan,\
                    '19036401040007526':np.nan,\
                    '210401293':np.nan,\
                    '219600236':np.nan,\
                    '317113148163123':np.nan,\
                    '19036401948872579':np.nan,\
                    '817284122210001':np.nan,\
                    '853290057220419':np.nan,\
                    '401110048164051':np.nan,\
                    }
    df2.loc[:,'对手卡号'].replace(third_party,inplace=True)
    return df2

#如果‘对手卡号’15位的卡号与17位的卡号后15位完全相同，那么将15位的卡号替换成17位的卡号
def replace_card1517(df2):
    df4 = df2[['对手卡号','对手户名']].copy()
    df4.drop_duplicates(keep='first',inplace=True)
    df4.dropna(axis=0, how='any',inplace=True)
    df4 = df4[df4['对手户名'].str.contains('公司')]
    grouped = df4['对手卡号'].groupby(df4['对手户名'])
    card_totalcount = grouped.count()
    df3 =card_totalcount[card_totalcount.values==2]
    if df3.empty is False:
        key_list = list(df3.keys())
        df4 = df4[df4['对手户名'].isin(key_list)]
        df4 = df4.sort_values(by=['对手户名','对手卡号'])
        df4.reset_index(drop= True,inplace=True)
        dict1 ={}
        df4['后15位'] = df4['对手卡号'].str[-15:]
        i = 0 
        while i < (len(df4)-1):  
            j = i + 1        
            while j < (len(df4)):
                if (df4.iloc[i,2] == df4.iloc[j,2])&(df4.iloc[i,1] == df4.iloc[j,1]):
                    if ((len(df4.iloc[i,0])) == 17) & ((len(df4.iloc[j,0])) == 15):
                        dict1[df4.iloc[j,0]] = df4.iloc[i,0]
                    elif ((len(df4.iloc[i,0])) == 15) & ((len(df4.iloc[j,0])) == 17):
                        dict1[df4.iloc[i,0]] = df4.iloc[j,0]
                j += 1
            i += 1
        if dict1:
            df2.loc[:,['交易卡号','对手卡号']].replace(dict1,inplace=True)
    return df2

# 去重2：开始对交易双方的重复项进行去重（根据卡号）
def remove_duplicate(df2,time_1):
    #获取主账号,删除主账号为nan
    df4 = df2['交易卡号'].drop_duplicates()
    df4.dropna(how = 'any',axis = 0,inplace=True)
    primary_card = df4.values.tolist()
    
    df2['编号'] = df2.index
    #将有可能产生重复的数据加入box1
    box1 = df2[df2['交易卡号'].isin(primary_card) & df2['对手卡号'].isin(primary_card)]
    if box1.empty is True:
        df2.drop(['编号'],axis=1,inplace=True)
        print('没有重复项！')
        print()
        return df2
    box1 = box1[box1['交易卡号'] != box1['对手卡号']]
    if box1.empty is True:
        df2.drop(['编号'],axis=1,inplace=True)
        print('没有重复项！')
        print()
        return df2
    #先按照日期时间升序，再把相同时间的'出'放在'进'前面
    box1 = box1.sort_values(by =['日期时间','借贷方向'],ascending= [True,True])
    box1.reset_index(drop= True,inplace=True)

    #重复行的索引
    list_index = []
    
    i = 0
    line_num = len(box1)
    while i < (line_num-1):
        j = i + 1
        if box1.loc[i,'借贷方向'] == '出':
            cardA = box1.loc[i,'交易卡号']
            cardB = box1.loc[i,'对手卡号']
            trade_money = box1.loc[i,'交易金额']
            trade_date_time = box1.loc[i,'日期时间']
            trade_date_time_end = trade_date_time + datetime.timedelta(seconds = time_1)    
            while j < line_num:
                if (box1.loc[j,'日期时间'] > trade_date_time_end) or (box1.loc[j,'编号'] in list_index):
                    break
                else:
                    if (box1.loc[j,'借贷方向'] == '进'):
                        card1 = box1.loc[j,'交易卡号']
                        card2 = box1.loc[j,'对手卡号']
                        trade_money2 = box1.loc[j,'交易金额']
                        if (card1 == cardB) and (card2 == cardA) and (trade_money2 == trade_money):
                            list_index.append(box1.loc[j,'编号'])
                            break
                        else:
                            j += 1
                    else:        
                        j += 1
        i += 1
    #依据索引删除重复行
    df2.drop(index=list_index,inplace=True)   

    df2.reset_index(drop= True,inplace=True)
    df2.drop(['编号'],axis=1,inplace=True)
    
    print('交易双方已去重{0}行。'.format(len(list_index)))
    print()
    print('数据表还剩{0}行。'.format(len(df)))
    print()
    return df2

# 清洗：依据基础库的卡与银行对应，填充银行
def fill_bank(df2):
    card_yinhang = {'卡标识':['102033',	'103000',	'185720',	'303781',	'400937',	'400938',	'400939',	'400940',	'400941',	'400942',	'421437',	'424106',	'424107',	'424108',	'424109',	'424110',	'424111',	'424902',	'431502',	'451291',	'458060',	'491020',	'491040',	'493427',	'493878',	'512695',	'512732',	'522153',	'530980',	'532420',	'532430',	'535910',	'535918',	'537830',	'540297',	'540838',	'541068',	'541709',	'547628',	'547648',	'558808',	'558809',	'584016',	'603128',	'622842',	'682900',	'695800',	'843010',	'843360',	'843420',	'843610',	'843730',	'843800',	'843850',	'843900',	'870000',	'870100',	'870300',	'870400',	'870500',	'888000',	'955880',	'955881',	'955882',	'955888',	'911121',	'418152',	'439229',	'552580',	'552581',	'552582',	'552583',	'552585',	'552586',	'552588',	'552589',	'645621',	'439228',	'356390',	'356391',	'356392',	'356827',	'356828',	'356829',	'356830',	'356833',	'356835',	'356837',	'356838',	'356839',	'356840',	'356850',	'356851',	'356852',	'356856',	'356857',	'356858',	'356859',	'356868',	'356869',	'356879',	'356880',	'356881',	'356882',	'356885',	'356886',	'356887',	'356888',	'356889',	'356890',	'356895',	'356896',	'356899',	'360883',	'360884',	'370246',	'370247',	'370248',	'370249',	'370267',	'370285',	'370286',	'370287',	'370289',	'374738',	'374739',	'376966',	'376968',	'376969',	'377152',	'377153',	'377155',	'377158',	'377187',	'377677',	'400360',	'402658',	'402673',	'402674',	'402791',	'403361',	'403391',	'403392',	'403393',	'404117',	'404118',	'404119',	'404120',	'404121',	'404157',	'404158',	'404159',	'404171',	'404172',	'404173',	'404174',	'404738',	'404739',	'405512',	'406252',	'406254',	'406365',	'406366',	'407405',	'409665',	'409666',	'409667',	'409668',	'409669',	'409670',	'409671',	'409672',	'410062',	'412962',	'412963',	'415599',	'415752',	'415753',	'421317',	'421349',	'421393',	'421865',	'421869',	'421870',	'421871',	'422160',	'422161',	'425862',	'427010',	'427018',	'427019',	'427020',	'427028',	'427029',	'427030',	'427038',	'427039',	'427062',	'427064',	'427570',	'427571',	'428911',	'433666',	'433667',	'433668',	'433669',	'433670',	'433671',	'433680',	'434061',	'434062',	'434910',	'435744',	'435745',	'436718',	'436728',	'436738',	'436742',	'436745',	'436748',	'436768',	'436769',	'438088',	'438125',	'438126',	'438588',	'438589',	'438600',	'439188',	'439225',	'439226',	'439227',	'442729',	'442730',	'451289',	'451290',	'451804',	'451810',	'451811',	'453242',	'456351',	'456418',	'458071',	'458123',	'458124',	'458441',	'461982',	'463758',	'464580',	'464581',	'468203',	'472067',	'472068',	'479228',	'479229',	'481699',	'483536',	'486466',	'486493',	'486494',	'486497',	'486861',	'487013',	'489592',	'489734',	'489735',	'489736',	'491031',	'491032',	'491034',	'491035',	'491036',	'491037',	'491038',	'498451',	'504923',	'510529',	'512315',	'512316',	'512411',	'512412',	'512425',	'512431',	'512466',	'513685',	'514027',	'514906',	'514957',	'514958',	'515672',	'517636',	'517650',	'518212',	'518364',	'518377',	'518378',	'518379',	'518474',	'518475',	'518476',	'518710',	'518718',	'519412',	'519413',	'519498',	'519961',	'520082',	'520083',	'520108',	'520131',	'520152',	'520169',	'520194',	'520382',	'521302',	'521899',	'522001',	'522964',	'523036',	'523952',	'523959',	'524011',	'524031',	'524047',	'524070',	'524090',	'524091',	'524094',	'524374',	'524864',	'524865',	'525498',	'525745',	'525746',	'525998',	'526410',	'526836',	'526855',	'527414',	'528020',	'528057',	'528708',	'528709',	'528856',	'528931',	'528948',	'530970',	'530990',	'531659',	'531693',	'532450',	'532458',	'539867',	'539868',	'543098',	'543159',	'544033',	'544210',	'544243',	'544887',	'545217',	'545392',	'545393',	'545431',	'545447',	'545619',	'545620',	'545621',	'545623',	'545947',	'545948',	'547766',	'548259',	'548478',	'548738',	'548838',	'548844',	'548943',	'549633',	'550213',	'552245',	'552288',	'552398',	'552534',	'552587',	'552599',	'552742',	'552794',	'552801',	'552853',	'553131',	'553161',	'553242',	'556610',	'556617',	'557080',	'558360',	'558730',	'558868',	'558869',	'558894',	'558895',	'558916',	'559051',	'566666',	'589970',	'601382',	'601428',	'602907',	'602969',	'603367',	'603445',	'603506',	'603601',	'603602',	'603694',	'603708',	'620009',	'620010',	'620011',	'620013',	'620015',	'620019',	'620021',	'620024',	'620025',	'620026',	'620027',	'620030',	'620031',	'620035',	'620037',	'620038',	'620039',	'620040',	'620043',	'620046',	'620048',	'620050',	'620054',	'620058',	'620059',	'620060',	'620061',	'620062',	'620068',	'620070',	'620072',	'620079',	'620082',	'620085',	'620086',	'620088',	'620091',	'620094',	'620101',	'620103',	'620105',	'620106',	'620107',	'620108',	'620114',	'620118',	'620120',	'620123',	'620124',	'620125',	'620126',	'620129',	'620132',	'620142',	'620143',	'620146',	'620148',	'620149',	'620152',	'620153',	'620183',	'620184',	'620185',	'620186',	'620187',	'620200',	'620202',	'620203',	'620204',	'620205',	'620206',	'620207',	'620208',	'620209',	'620210',	'620211',	'620220',	'620278',	'620302',	'620402',	'620403',	'620404',	'620405',	'620406',	'620407',	'620408',	'620409',	'620410',	'620411',	'620412',	'620500',	'620501',	'620502',	'620503',	'620512',	'620513',	'620514',	'620515',	'620516',	'620517',	'620518',	'620519',	'620520',	'620521',	'620522',	'620527',	'620528',	'620529',	'620530',	'620531',	'620532',	'620535',	'620537',	'620550',	'620561',	'620602',	'620604',	'620607',	'620609',	'620611',	'620612',	'620704',	'620706',	'620707',	'620708',	'620709',	'620710',	'620711',	'620712',	'620713',	'620714',	'620802',	'620812',	'620902',	'620904',	'620905',	'621001',	'621002',	'621003',	'621004',	'621005',	'621006',	'621008',	'621010',	'621011',	'621012',	'621013',	'621014',	'621015',	'621016',	'621017',	'621018',	'621019',	'621020',	'621021',	'621023',	'621024',	'621025',	'621026',	'621027',	'621028',	'621029',	'621030',	'621031',	'621032',	'621033',	'621034',	'621035',	'621036',	'621037',	'621038',	'621039',	'621040',	'621041',	'621042',	'621043',	'621044',	'621045',	'621049',	'621050',	'621053',	'621055',	'621057',	'621058',	'621060',	'621061',	'621062',	'621063',	'621064',	'621065',	'621066',	'621067',	'621068',	'621069',	'621070',	'621071',	'621072',	'621073',	'621074',	'621075',	'621076',	'621077',	'621078',	'621080',	'621081',	'621082',	'621083',	'621084',	'621086',	'621087',	'621088',	'621089',	'621090',	'621091',	'621095',	'621096',	'621097',	'621098',	'621099',	'621102',	'621103',	'621105',	'621106',	'621107',	'621200',	'621201',	'621202',	'621203',	'621204',	'621205',	'621206',	'621207',	'621208',	'621209',	'621210',	'621211',	'621212',	'621213',	'621215',	'621216',	'621217',	'621220',	'621221',	'621222',	'621223',	'621224',	'621225',	'621226',	'621227',	'621228',	'621229',	'621230',	'621231',	'621232',	'621233',	'621234',	'621235',	'621237',	'621238',	'621239',	'621240',	'621242',	'621244',	'621245',	'621247',	'621248',	'621249',	'621251',	'621252',	'621253',	'621254',	'621255',	'621256',	'621257',	'621258',	'621259',	'621263',	'621264',	'621266',	'621267',	'621268',	'621269',	'621270',	'621271',	'621272',	'621273',	'621274',	'621277',	'621279',	'621280',	'621281',	'621282',	'621283',	'621284',	'621285',	'621286',	'621287',	'621288',	'621289',	'621290',	'621291',	'621292',	'621293',	'621294',	'621295',	'621296',	'621297',	'621298',	'621299',	'621300',	'621301',	'621302',	'621303',	'621304',	'621305',	'621306',	'621307',	'621309',	'621310',	'621311',	'621313',	'621315',	'621317',	'621324',	'621325',	'621327',	'621328',	'621330',	'621331',	'621332',	'621333',	'621334',	'621335',	'621336',	'621337',	'621339',	'621340',	'621341',	'621342',	'621343',	'621344',	'621345',	'621348',	'621349',	'621351',	'621352',	'621354',	'621359',	'621360',	'621361',	'621362',	'621363',	'621364',	'621366',	'621367',	'621369',	'621370',	'621371',	'621372',	'621374',	'621375',	'621376',	'621377',	'621378',	'621379',	'621388',	'621390',	'621391',	'621392',	'621394',	'621395',	'621396',	'621402',	'621403',	'621404',	'621405',	'621406',	'621407',	'621408',	'621409',	'621410',	'621411',	'621412',	'621413',	'621414',	'621415',	'621416',	'621417',	'621418',	'621419',	'621420',	'621422',	'621423',	'621428',	'621433',	'621434',	'621436',	'621437',	'621439',	'621440',	'621441',	'621442',	'621443',	'621446',	'621448',	'621449',	'621452',	'621453',	'621455',	'621456',	'621457',	'621458',	'621459',	'621460',	'621461',	'621462',	'621463',	'621464',	'621465',	'621466',	'621467',	'621468',	'621469',	'621480',	'621481',	'621482',	'621483',	'621484',	'621485',	'621486',	'621487',	'621488',	'621489',	'621490',	'621491',	'621492',	'621495',	'621496',	'621497',	'621498',	'621499',	'621502',	'621511',	'621515',	'621516',	'621517',	'621518',	'621519',	'621520',	'621521',	'621522',	'621523',	'621525',	'621526',	'621527',	'621528',	'621529',	'621530',	'621531',	'621532',	'621533',	'621536',	'621538',	'621539',	'621557',	'621558',	'621559',	'621560',	'621561',	'621562',	'621563',	'621566',	'621568',	'621569',	'621577',	'621578',	'621579',	'621580',	'621585',	'621588',	'621589',	'621590',	'621591',	'621592',	'621598',	'621599',	'621600',	'621601',	'621602',	'621603',	'621604',	'621605',	'621606',	'621607',	'621608',	'621609',	'621610',	'621611',	'621612',	'621613',	'621614',	'621615',	'621616',	'621617',	'621618',	'621619',	'621620',	'621621',	'621622',	'621624',	'621625',	'621626',	'621633',	'621636',	'621638',	'621640',	'621642',	'621645',	'621647',	'621648',	'621649',	'621651',	'621654',	'621655',	'621657',	'621660',	'621661',	'621662',	'621663',	'621665',	'621666',	'621667',	'621668',	'621669',	'621670',	'621671',	'621672',	'621673',	'621674',	'621688',	'621690',	'621691',	'621694',	'621696',	'621700',	'621701',	'621719',	'621720',	'621721',	'621722',	'621723',	'621724',	'621725',	'621726',	'621727',	'621728',	'621730',	'621731',	'621732',	'621733',	'621734',	'621735',	'621738',	'621739',	'621740',	'621741',	'621742',	'621743',	'621744',	'621745',	'621746',	'621747',	'621748',	'621749',	'621750',	'621751',	'621752',	'621753',	'621754',	'621755',	'621756',	'621757',	'621758',	'621759',	'621760',	'621761',	'621762',	'621763',	'621764',	'621765',	'621766',	'621767',	'621768',	'621770',	'621771',	'621772',	'621773',	'621775',	'621777',	'621778',	'621780',	'621781',	'621782',	'621783',	'621784',	'621785',	'621786',	'621787',	'621788',	'621789',	'621790',	'621791',	'621792',	'621793',	'621795',	'621796',	'621797',	'621798',	'621799',	'621804',	'621807',	'621813',	'621814',	'621817',	'621901',	'621903',	'621904',	'621905',	'621906',	'621907',	'621908',	'621909',	'621910',	'621911',	'621912',	'621913',	'621914',	'621915',	'621977',	'622002',	'622003',	'622004',	'622005',	'622006',	'622007',	'622008',	'622010',	'622011',	'622012',	'622013',	'622015',	'622016',	'622017',	'622018',	'622019',	'622020',	'622102',	'622103',	'622104',	'622105',	'622110',	'622111',	'622114',	'622126',	'622127',	'622128',	'622129',	'622130',	'622131',	'622132',	'622133',	'622134',	'622135',	'622136',	'622137',	'622138',	'622139',	'622140',	'622141',	'622143',	'622145',	'622146',	'622147',	'622148',	'622149',	'622150',	'622151',	'622152',	'622153',	'622154',	'622155',	'622156',	'622157',	'622158',	'622159',	'622161',	'622162',	'622163',	'622164',	'622165',	'622166',	'622167',	'622168',	'622169',	'622170',	'622171',	'622172',	'622173',	'622176',	'622177',	'622178',	'622179',	'622181',	'622184',	'622188',	'622199',	'622200',	'622202',	'622203',	'622206',	'622208',	'622210',	'622211',	'622212',	'622213',	'622214',	'622215',	'622218',	'622220',	'622223',	'622224',	'622225',	'622227',	'622228',	'622229',	'622230',	'622231',	'622232',	'622233',	'622234',	'622235',	'622236',	'622237',	'622238',	'622239',	'622240',	'622245',	'622246',	'622250',	'622251',	'622252',	'622253',	'622254',	'622255',	'622256',	'622257',	'622258',	'622259',	'622260',	'622261',	'622262',	'622265',	'622266',	'622267',	'622268',	'622269',	'622270',	'622271',	'622272',	'622273',	'622274',	'622275',	'622276',	'622277',	'622278',	'622279',	'622280',	'622281',	'622282',	'622283',	'622284',	'622286',	'622287',	'622288',	'622289',	'622290',	'622291',	'622292',	'622293',	'622294',	'622295',	'622296',	'622297',	'622298',	'622300',	'622301',	'622302',	'622303',	'622304',	'622305',	'622306',	'622307',	'622308',	'622309',	'622310',	'622311',	'622312',	'622313',	'622314',	'622315',	'622316',	'622317',	'622318',	'622319',	'622320',	'622321',	'622322',	'622323',	'622324',	'622325',	'622327',	'622328',	'622329',	'622331',	'622332',	'622333',	'622335',	'622336',	'622337',	'622338',	'622339',	'622340',	'622341',	'622342',	'622343',	'622345',	'622346',	'622347',	'622348',	'622349',	'622350',	'622351',	'622352',	'622353',	'622354',	'622355',	'622356',	'622358',	'622359',	'622360',	'622361',	'622362',	'622363',	'622365',	'622366',	'622367',	'622368',	'622369',	'622370',	'622371',	'622372',	'622373',	'622375',	'622376',	'622377',	'622378',	'622379',	'622381',	'622382',	'622383',	'622384',	'622385',	'622386',	'622387',	'622388',	'622389',	'622391',	'622392',	'622393',	'622394',	'622395',	'622396',	'622397',	'622398',	'622399',	'622400',	'622402',	'622403',	'622404',	'622406',	'622407',	'622409',	'622410',	'622411',	'622412',	'622413',	'622414',	'622415',	'622418',	'622420',	'622421',	'622422',	'622423',	'622425',	'622426',	'622427',	'622428',	'622429',	'622432',	'622433',	'622434',	'622435',	'622436',	'622439',	'622440',	'622441',	'622442',	'622443',	'622444',	'622447',	'622448',	'622449',	'622450',	'622451',	'622452',	'622453',	'622455',	'622456',	'622459',	'622462',	'622463',	'622465',	'622466',	'622467',	'622468',	'622469',	'622470',	'622471',	'622472',	'622475',	'622476',	'622477',	'622478',	'622479',	'622480',	'622481',	'622482',	'622483',	'622484',	'622485',	'622486',	'622487',	'622488',	'622489',	'622490',	'622491',	'622492',	'622493',	'622495',	'622496',	'622498',	'622499',	'622500',	'622502',	'622504',	'622505',	'622506',	'622508',	'622509',	'622510',	'622511',	'622513',	'622516',	'622517',	'622518',	'622519',	'622520',	'622521',	'622522',	'622523',	'622525',	'622526',	'622531',	'622532',	'622535',	'622536',	'622538',	'622539',	'622546',	'622547',	'622548',	'622549',	'622550',	'622555',	'622556',	'622557',	'622558',	'622559',	'622560',	'622561',	'622562',	'622563',	'622565',	'622566',	'622567',	'622568',	'622569',	'622570',	'622575',	'622576',	'622577',	'622578',	'622579',	'622580',	'622581',	'622582',	'622588',	'622595',	'622596',	'622597',	'622598',	'622599',	'622600',	'622601',	'622602',	'622603',	'622604',	'622605',	'622606',	'622609',	'622611',	'622613',	'622615',	'622616',	'622617',	'622618',	'622619',	'622620',	'622621',	'622622',	'622623',	'622625',	'622626',	'622630',	'622631',	'622632',	'622633',	'622636',	'622637',	'622638',	'622644',	'622648',	'622650',	'622651',	'622655',	'622656',	'622657',	'622658',	'622659',	'622660',	'622661',	'622662',	'622663',	'622664',	'622665',	'622666',	'622667',	'622668',	'622669',	'622670',	'622671',	'622672',	'622673',	'622674',	'622675',	'622676',	'622677',	'622678',	'622679',	'622680',	'622681',	'622682',	'622683',	'622684',	'622685',	'622686',	'622687',	'622688',	'622689',	'622690',	'622691',	'622692',	'622696',	'622698',	'622700',	'622703',	'622706',	'622707',	'622708',	'622715',	'622717',	'622722',	'622725',	'622728',	'622740',	'622750',	'622751',	'622752',	'622753',	'622754',	'622755',	'622756',	'622757',	'622758',	'622759',	'622760',	'622761',	'622762',	'622763',	'622764',	'622765',	'622770',	'622771',	'622772',	'622775',	'622777',	'622778',	'622785',	'622788',	'622789',	'622790',	'622798',	'622801',	'622802',	'622806',	'622808',	'622809',	'622810',	'622811',	'622812',	'622813',	'622815',	'622816',	'622817',	'622818',	'622820',	'622821',	'622822',	'622823',	'622824',	'622825',	'622826',	'622827',	'622828',	'622829',	'622830',	'622836',	'622837',	'622838',	'622840',	'622841',	'622843',	'622844',	'622845',	'622846',	'622847',	'622848',	'622849',	'622851',	'622852',	'622853',	'622855',	'622856',	'622857',	'622858',	'622859',	'622860',	'622861',	'622862',	'622864',	'622865',	'622866',	'622867',	'622868',	'622869',	'622870',	'622871',	'622873',	'622876',	'622877',	'622878',	'622879',	'622880',	'622881',	'622882',	'622884',	'622885',	'622886',	'622888',	'622889',	'622891',	'622892',	'622893',	'622895',	'622897',	'622898',	'622899',	'622900',	'622901',	'622902',	'622903',	'622904',	'622906',	'622908',	'622909',	'622916',	'622918',	'622919',	'622920',	'622921',	'622922',	'622926',	'622927',	'622928',	'622929',	'622930',	'622931',	'622932',	'622933',	'622935',	'622936',	'622937',	'622938',	'622939',	'622940',	'622941',	'622942',	'622943',	'622944',	'622945',	'622946',	'622947',	'622948',	'622949',	'622950',	'622951',	'622952',	'622953',	'622954',	'622955',	'622957',	'622958',	'622959',	'622960',	'622961',	'622962',	'622963',	'622966',	'622967',	'622968',	'622970',	'622971',	'622972',	'622973',	'622974',	'622976',	'622977',	'622978',	'622979',	'622980',	'622981',	'622982',	'622983',	'622985',	'622986',	'622987',	'622988',	'622989',	'622990',	'622991',	'622992',	'622993',	'622994',	'622995',	'622996',	'622997',	'622998',	'622999',	'623000',	'623001',	'623002',	'623003',	'623006',	'623007',	'623008',	'623010',	'623011',	'623012',	'623013',	'623014',	'623015',	'623016',	'623017',	'623018',	'623019',	'623020',	'623021',	'623022',	'623023',	'623024',	'623025',	'623026',	'623027',	'623028',	'623029',	'623030',	'623031',	'623032',	'623033',	'623034',	'623035',	'623036',	'623037',	'623038',	'623039',	'623040',	'623041',	'623042',	'623043',	'623048',	'623051',	'623055',	'623058',	'623059',	'623060',	'623061',	'623062',	'623063',	'623064',	'623065',	'623066',	'623067',	'623068',	'623069',	'623070',	'623071',	'623072',	'623073',	'623075',	'623076',	'623077',	'623078',	'623079',	'623081',	'623083',	'623084',	'623085',	'623086',	'623087',	'623088',	'623089',	'623090',	'623091',	'623092',	'623093',	'623095',	'623096',	'623098',	'623099',	'623100',	'623101',	'623102',	'623103',	'623105',	'623106',	'623107',	'623108',	'623109',	'623110',	'623111',	'623113',	'623115',	'623116',	'623118',	'623119',	'623120',	'623121',	'623123',	'623125',	'623128',	'623129',	'623131',	'623132',	'623133',	'623135',	'623138',	'623139',	'623151',	'623152',	'623153',	'623155',	'623156',	'623157',	'623158',	'623159',	'623160',	'623161',	'623162',	'623165',	'623166',	'623168',	'623170',	'623171',	'623172',	'623173',	'623177',	'623178',	'623181',	'623182',	'623183',	'623185',	'623186',	'623188',	'623189',	'623190',	'623196',	'623199',	'623200',	'623201',	'623202',	'623203',	'623205',	'623206',	'623207',	'623208',	'623209',	'623211',	'623218',	'623219',	'623251',	'623300',	'623301',	'623302',	'623303',	'623304',	'623307',	'623308',	'623309',	'623310',	'623311',	'623312',	'623313',	'623314',	'623318',	'623323',	'623324',	'623328',	'623331',	'623334',	'623335',	'623336',	'623337',	'623338',	'623339',	'623341',	'623347',	'623348',	'623400',	'623500',	'623506',	'623510',	'623516',	'623518',	'623521',	'623523',	'623602',	'623688',	'623700',	'623803',	'623901',	'624000',	'624100',	'624200',	'624301',	'624303',	'624306',	'624320',	'624321',	'624322',	'624323',	'624324',	'624325',	'624402',	'625001',	'625002',	'625003',	'625004',	'625006',	'625007',	'625008',	'625009',	'625010',	'625011',	'625012',	'625013',	'625014',	'625016',	'625017',	'625018',	'625019',	'625020',	'625021',	'625022',	'625023',	'625024',	'625025',	'625026',	'625027',	'625028',	'625029',	'625031',	'625032',	'625033',	'625034',	'625035',	'625036',	'625039',	'625040',	'625042',	'625044',	'625046',	'625048',	'625050',	'625053',	'625055',	'625058',	'625060',	'625062',	'625063',	'625071',	'625072',	'625074',	'625075',	'625076',	'625077',	'625078',	'625079',	'625080',	'625082',	'625083',	'625084',	'625085',	'625086',	'625087',	'625088',	'625090',	'625091',	'625092',	'625093',	'625095',	'625096',	'625098',	'625099',	'625101',	'625103',	'625104',	'625106',	'625107',	'625110',	'625111',	'625112',	'625113',	'625114',	'625115',	'625116',	'625119',	'625120',	'625123',	'625124',	'625125',	'625127',	'625128',	'625131',	'625132',	'625135',	'625136',	'625139',	'625140',	'625141',	'625143',	'625145',	'625147',	'625154',	'625158',	'625178',	'625179',	'625186',	'625196',	'625198',	'625220',	'625243',	'625244',	'625288',	'625320',	'625330',	'625331',	'625332',	'625333',	'625336',	'625337',	'625338',	'625339',	'625350',	'625351',	'625352',	'625360',	'625361',	'625362',	'625363',	'625500',	'625502',	'625503',	'625506',	'625516',	'625519',	'625526',	'625529',	'625568',	'625577',	'625588',	'625598',	'625652',	'625700',	'625708',	'625709',	'625800',	'625802',	'625803',	'625804',	'625805',	'625806',	'625807',	'625808',	'625809',	'625810',	'625814',	'625817',	'625819',	'625823',	'625825',	'625826',	'625827',	'625828',	'625829',	'625831',	'625836',	'625839',	'625840',	'625841',	'625842',	'625865',	'625866',	'625888',	'625899',	'625900',	'625901',	'625902',	'625903',	'625904',	'625905',	'625906',	'625907',	'625908',	'625909',	'625910',	'625911',	'625912',	'625913',	'625914',	'625915',	'625916',	'625917',	'625918',	'625919',	'625920',	'625921',	'625922',	'625924',	'625925',	'625926',	'625927',	'625928',	'625929',	'625930',	'625931',	'625932',	'625933',	'625934',	'625939',	'625940',	'625942',	'625943',	'625946',	'625950',	'625952',	'625953',	'625955',	'625956',	'625957',	'625958',	'625959',	'625960',	'625961',	'625962',	'625963',	'625964',	'625965',	'625966',	'625967',	'625968',	'625969',	'625970',	'625971',	'625972',	'625973',	'625975',	'625976',	'625977',	'625978',	'625979',	'625980',	'625981',	'625986',	'625987',	'625988',	'625989',	'625993',	'625995',	'625996',	'625997',	'625998',	'627066',	'627067',	'627068',	'627069',	'628200',	'628201',	'628202',	'628203',	'628204',	'628205',	'628206',	'628207',	'628208',	'628209',	'628210',	'628211',	'628212',	'628213',	'628214',	'628216',	'628217',	'628218',	'628219',	'628220',	'628221',	'628222',	'628223',	'628224',	'628226',	'628227',	'628228',	'628229',	'628230',	'628231',	'628232',	'628233',	'628234',	'628235',	'628236',	'628237',	'628238',	'628239',	'628242',	'628248',	'628250',	'628251',	'628252',	'628253',	'628255',	'628257',	'628258',	'628259',	'628260',	'628261',	'628262',	'628263',	'628266',	'628267',	'628268',	'628269',	'628270',	'628271',	'628272',	'628273',	'628275',	'628278',	'628279',	'628280',	'628281',	'628282',	'628283',	'628285',	'628286',	'628287',	'628288',	'628289',	'628290',	'628291',	'628293',	'628295',	'628296',	'628297',	'628299',	'628300',	'628301',	'628303',	'628305',	'628306',	'628307',	'628308',	'628309',	'628310',	'628311',	'628312',	'628313',	'628315',	'628316',	'628317',	'628318',	'628319',	'628321',	'628322',	'628323',	'628328',	'628329',	'628330',	'628331',	'628332',	'628333',	'628336',	'628339',	'628351',	'628355',	'628358',	'628359',	'628362',	'628366',	'628367',	'628368',	'628369',	'628370',	'628371',	'628372',	'628378',	'628379',	'628381',	'628382',	'628386',	'628388',	'628389',	'628391',	'628392',	'628397',	'628398',	'683970',	'685800',	'690755',	'694301',	'900000',	'900010',	'900105',	'900205',	'909810',	'920000',	'921000',	'921001',	'921002',	'940001',	'940002',	'940003',	'940006',	'940008',	'940012',	'940013',	'940015',	'940016',	'940017',	'940018',	'940020',	'940021',	'940022',	'940023',	'940025',	'940027',	'940029',	'940031',	'940032',	'940034',	'940035',	'940037',	'940038',	'940039',	'940040',	'940041',	'940042',	'940043',	'940044',	'940046',	'940047',	'940048',	'940049',	'940050',	'940051',	'940053',	'940054',	'940055',	'940056',	'940057',	'940058',	'940060',	'940061',	'940062',	'940063',	'940065',	'940066',	'940068',	'940069',	'940070',	'940071',	'940072',	'940073',	'940074',	'940076',	'955100',	'955590',	'955591',	'955592',	'955593',	'966666',	'968807',	'968808',	'968809',	'984301',	'984303',	'985262',	'990027',	'990871',	'998800',	'998801',	'998802',	'999999'],\
                '银行':['广东发展银行',	'农业银行',	'昆明农联社',	'中国光大银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中信实业银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'华夏',	'中国银行',	'工商银行',	'农业银行',	'交通银行',	'广东发展银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'工商银行',	'建设银行',	'建设银行',	'农业银行',	'农业银行',	'交通银行',	'中国银行',	'中国银行',	'中国银行',	'广东发展银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'深圳农联社',	'南京市商业银行',	'农业银行',	'上海银行',	'南通商业银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'贵阳市商业银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'广东发展银行',	'上海浦东发展银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'上海银行',	'上海银行',	'上海银行',	'上海银行',	'中国银行',	'中国银行',	'光大银行',	'光大银行',	'中国光大银行',	'中国光大银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国工商银行',	'中国工商银行',	'中国工商银行',	'中国工商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'中国建设银行',	'中国建设银行',	'中国建设银行',	'中国工商银行',	'中国工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'招商银行信用卡中心',	'招商银行信用卡中心',	'中国工商银行',	'中国工商银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'浦发银行信用卡中心',	'中国银行',	'中信银行信用卡中心',	'招商银行',	'上海银行',	'上海银行',	'工商银行',	'农业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'浦东发展银行',	'浦东发展银行',	'交通银行',	'中国光大银行',	'中国光大银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'民生银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'招商银行',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'民生银行',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'北京银行',	'建设银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'北京银行',	'北京银行',	'中国光大银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'民生银行',	'民生银行',	'广发银行股份有限公司',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行',	'中信银行',	'中信银行',	'建设银行',	'建设银行',	'交通银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国建设银行',	'建设银行',	'中国建设银行',	'建设银行',	'中国建设银行',	'中国建设银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'中国银行',	'工商银行',	'工商银行',	'兴业银行',	'兴业银行',	'上海银行',	'招商银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'招商银行',	'中信银行',	'中信银行',	'兴业银行',	'兴业银行',	'工商银行',	'工商银行',	'工商银行',	'建设银行',	'中国银行',	'浦东发展银行',	'工商银行',	'交通银行',	'交通银行',	'工商银行',	'兴业银行',	'农业银行',	'民生银行',	'民生银行',	'招商银行',	'民生银行',	'民生银行',	'招商银行',	'招商银行',	'中国光大银行',	'平安银行（深圳发展银行）股份有限公司',	'上海银行',	'兴业银行',	'兴业银行',	'光大银行',	'兴业银行',	'广发银行股份有限公司',	'中国建设银行',	'工商银行',	'工商银行',	'工商银行',	'建设银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'浦东发展银行',	'江苏银行',	'工商银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'招商银行',	'宁波银行',	'民生银行',	'工商银行',	'农业银行',	'中信银行信用卡中心',	'中国银行',	'中国银行',	'浦东发展银行',	'民生银行',	'浦东发展银行',	'中信银行信用卡中心',	'广发银行股份有限公司',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'农业银行',	'农业银行',	'上海银行',	'上海银行',	'农业银行',	'农业银行',	'中信银行信用卡中心',	'上海银行',	'广发银行股份有限公司',	'交通银行',	'宁波银行',	'广发银行股份有限公司',	'招商银行',	'交通银行',	'北京银行',	'交通银行',	'兴业银行',	'民生银行',	'华夏银行',	'招商银行',	'上海银行',	'工商银行',	'兴业银行',	'中国光大银行',	'工商银行',	'建设银行',	'中国工商银行',	'中国银行',	'中国银行',	'工商银行',	'中国银行',	'中国银行',	'浦东发展银行',	'建设银行',	'工商银行',	'平安银行（深圳发展银行）股份有限公司',	'兴业银行',	'平安银行（深圳发展银行）股份有限公司',	'兴业银行',	'华夏银行',	'华夏银行',	'中国工商银行',	'广发银行股份有限公司',	'民生银行',	'工商银行',	'工商银行',	'平安银行（深圳发展银行）股份有限公司',	'中国建设银行',	'中国建设银行',	'中国建设银行',	'华夏银行',	'华夏银行',	'工商银行',	'中国光大银行',	'建设银行',	'中国工商银行',	'中国农业银行贷记卡',	'中国建设银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'中国银行',	'工商银行',	'中国农业银行贷记卡',	'兴业银行',	'上海银行',	'广发银行股份有限公司',	'中国工商银行',	'兴业银行',	'中国工商银行',	'建设银行',	'民生银行',	'兴业银行',	'招商银行',	'招商银行',	'农业银行',	'中国银行',	'广发银行股份有限公司',	'中国建设银行',	'交通银行',	'中国银行',	'民生银行',	'建设银行',	'民生银行',	'中信银行信用卡中心',	'中国建设银行',	'工商银行',	'农业银行',	'中国银行',	'中国银行',	'广发银行股份有限公司',	'中国建设银行',	'中信银行信用卡中心',	'中国建设银行',	'盛京银行',	'建设银行',	'中国银行',	'交通银行',	'平安银行（深圳发展银行）',	'北京银行',	'杭州商业银行',	'广州银行股份有限公司',	'苏州市商业银行',	'徽商银行合肥分行',	'绍兴银行股份有限公司',	'常熟农村商业银行',	'大连银行',	'越南西贡商业银行',	'平安银行（深圳发展银行）（借记卡）',	'BC卡公司',	'交通银行香港分行',	'新加坡星网电子付款私人有限公司',	'中国银行(马来西亚)',	'交通银行',	'Royal Bank Open Stock Company',	'中国银行（澳大利亚）',	'中国银行（澳大利亚）',	'BC卡公司',	'中国工商银行(亚洲)有限公司',	'BC卡公司',	'中国银行(马来西亚)',	'广发银行股份有限公司',	'中银通商务支付有限公司',	'BC卡公司',	'中国银行',	'福建海峡银行股份有限公司',	'工银河内',	'中银通商务支付有限公司',	'中国工商银行(亚洲)有限公司',	'中国工商银行布鲁塞尔分行',	'工商银行',	'农业银行',	'建设银行',	'中国银行',	'邮储银行',	'中国银盛',	'中国银盛',	'大丰银行有限公司',	'Commercial Bank of Dubai',	'中信银行',	'光大银行',	'中国工商银行',	'北京农村商业银行',	'Commercial Bank of Dubai',	'中国工商银行新加坡分行',	'中国工商银行(亚洲)有限公司',	'BC卡公司',	'The Bancorp Bank',	'BC卡公司',	'中国建设银行（亚洲）',	'Russian Standard Bank',	'中国工商银行法兰克福分行',	'柳州银行股份有限公司',	'BC卡公司',	'BC卡公司',	'中国工商银行澳门分行',	'BC卡公司',	'澳门通股份有限公司',	'The Mauritius Commercial Bank',	'BC卡公司',	'中国工商银行布鲁塞尔分行',	'中国工商银行米兰分行',	'中国工商银行法兰克福分行',	'中国工商银行马德里分行',	'中国工商银行米兰分行',	'Travelex',	'Travelex',	'中国工商银行澳门分行',	'中国工商银行加拿大分行',	'中国工商银行马德里分行',	'中国工商银行新加坡分行',	'中国工商银行阿拉木图子行',	'工商银行',	'中国银行（香港）',	'中国银行（香港）',	'大丰银行有限公司',	'大丰银行有限公司',	'集友银行',	'集友银行',	'南洋商业银行',	'南洋商业银行',	'中国银行澳门分行',	'中国银行澳门分行',	'BC卡公司',	'BC卡公司',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'宁波鄞州农村合作银行',	'宁波市农业银行',	'工商银行',	'工商银行',	'工商银行',	'中行宁波分行',	'中国银行',	'中银通商务支付有限公司',	'工商银行',	'浙江民泰商业银行',	'光大银行',	'长沙银行股份有限公司',	'招商银行',	'交通银行',	'上海银行',	'中信银行',	'邢台银行',	'邮政储蓄银行',	'浦东发展银行',	'中国银行澳门分行',	'澳门通股份有限公司',	'光大银行',	'澳门通股份有限公司',	'中银通商务支付有限公司',	'中国工商银行澳门分行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'BC卡公司',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'交通银行香港分行',	'BC卡公司',	'东营银行',	'上海银行',	'BC卡公司',	'新疆维吾尔自治区农村信用社联合',	'平凉市商业银行',	'BC卡公司',	'BC卡公司',	'湖北农信社',	'上饶市商业银行',	'星展银行',	'星展银行',	'昆明农联社',	'广东省农村信用社联合社',	'浙商银行',	'BC卡公司',	'河北省农村信用社联合社',	'BC卡公司',	'宁波鄞州农村合作银行',	'BC卡公司',	'辽宁省农村信用社联合社',	'BC卡公司',	'浙江稠州商业银行',	'昆仑银行股份有限公司',	'北京银行',	'BC卡公司',	'BC卡公司',	'四川省农村信用社联合社',	'上海商业银行',	'许昌银行',	'海南省农村信用社联合社',	'龙江银行',	'铁岭银行',	'BC卡公司',	'莫斯科人民储蓄银行',	'中国银行(香港)',	'南洋商业银行',	'集友银行',	'宝鸡商行',	'丝绸之路银行',	'蒙古郭勒姆特银行',	'上海银行',	'福建建瓯石狮村镇银行',	'越南西贡商业银行',	'凉山州商业银行',	'浙江省农村信用社联合社',	'友利银行(中国)有限公司',	'甘肃省农村信用社联合社',	'花旗银行(中国)有限公司',	'花旗银行(中国)有限公司',	'AEON信贷财务亚洲有限公司',	'青海省农村信用社联合社',	'北京农村商业银行',	'北京农村商业银行',	'北京农村商业银行',	'交通银行香港分行',	'自贡市商业银行',	'平顶山银行股份有限公司',	'新韩银行',	'天津滨海农村商业银行',	'朝阳银行',	'漯河银行',	'江苏银行',	'华侨银行（中国）',	'BC卡公司',	'建设银行',	'建设银行',	'建设银行',	'中国建设银行亚洲股份有限公司',	'中国建设银行澳门股份有限公司',	'乐山市商业银行',	'创兴银行有限公司',	'浙江民泰商业银行',	'盘锦市商业银行',	'遂宁市商业银行',	'保定银行',	'邮储银行',	'邮储银行',	'龙江银行',	'邮储银行',	'四川省农村信用社联合社',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'济宁银行股份有限公司',	'韩亚银行（中国）',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国银行',	'南洋商业银行（中国）',	'中银东京分行',	'晋商银行',	'湖北银行',	'BC卡公司',	'驻马店银行',	'华夏银行',	'晋中银行',	'可汗银行',	'工商银行',	'工商银行',	'工商银行',	'黑龙江省农村信用社联合社',	'重庆璧山工银村镇银行',	'浙江平湖工银村镇银行',	'中国银行马尼拉分行',	'大西洋银行股份有限公司',	'达州市商业银行',	'CSC',	'新乡市商业银行',	'秦皇岛银行股份有限公司',	'邢台银行股份有限公司',	'衡水银行',	'中国工商银行(印尼)',	'兰州银行股份有限公司',	'盛京银行',	'法国兴业银行（中国）',	'大西洋银行股份有限公司',	'中国银行雅加达分行',	'中国银行首尔分行',	'福建省农村信用社联合社',	'青岛银行',	'澳门商业银行',	'澳门商业银行',	'澳门商业银行',	'中国银行',	'Baiduri Bank Berhad',	'重庆农村商业银行',	'南京银行',	'库尔勒市商业银行',	'俄罗斯远东商业银行',	'沧州银行股份有限公司',	'福建海峡银行股份有限公司',	'渤海银行',	'南昌银行',	'湖北银行',	'周口银行',	'阳泉市商业银行',	'宜宾市商业银行',	'澳门BDA',	'大新银行（中国）',	'宁波银行',	'山西省农村信用社',	'工商银行',	'农业银行',	'中国银行',	'建设银行',	'邮储银行',	'招商银行',	'新疆维吾尔自治区农村信用社联合',	'工商银行',	'南洋商业银行（中国）',	'南洋商业银行（中国）',	'南洋商业银行（中国）',	'南洋商业银行（中国）',	'中国银行（澳大利亚）',	'中国银行（澳大利亚）',	'越南Vietcombank',	'长安银行',	'中国银行',	'永亨银行（中国）有限公司',	'招商银行香港分行',	'中国工商银行巴黎分行',	'格鲁吉亚 Invest Bank',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'三水珠江村镇银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'澳门BDA',	'雅安市商业银行',	'安阳银行',	'大华银行（中国）',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行曼谷分行',	'交通银行澳门分行',	'农业银行',	'商丘商行',	'新疆汇和银行',	'廊坊银行',	'廊坊银行',	'中国银行（澳大利亚）',	'中国银行（澳大利亚）',	'Royal Bank Open Stock Company',	'宁波东海银行',	'营口沿海银行',	'乌兹别克斯坦INFINBANK',	'浦东发展银行',	'浦东发展银行',	'BCEL',	'景德镇商业银行',	'哈密市商业银行',	'武汉农村商业银行',	'内蒙古自治区农村信用社联合式',	'江南农村商业银行',	'中国银行（澳大利亚）',	'华融湘江银行',	'工银泰国',	'工银河内',	'中国工商银行卢森堡分行',	'中国工商银行米兰分行',	'中国工商银行阿姆斯特丹',	'中国工商银行伦敦子行',	'中国工商银行阿拉木图子行',	'中国工商银行布鲁塞尔分行',	'中国工商银行马德里分行',	'中国工商银行巴黎分行',	'中国工商银行法兰克福分行',	'华融湘江银行',	'浦东发展银行',	'西藏银行',	'南阳村镇银行',	'中国银行（澳大利亚）',	'中国银行曼谷分行',	'东营莱商村镇银行',	'工商银行',	'莱商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'东亚银行澳门分行',	'柳州银行股份有限公司',	'张家口市商业银行',	'中国工商银行阿拉木图子行',	'昆明商业银行',	'上饶银行',	'宁夏银行',	'宁波银行',	'青岛银行',	'北京银行',	'沧州银行',	'中国工商银行（巴西）',	'中国工商银行金边分行',	'中国工商银行万象分行',	'中国工商银行金边分行',	'交通银行香港分行',	'泉州银行',	'东莞市商业银行',	'恒生银行',	'恒生银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'长沙银行股份有限公司',	'长安银行',	'辽宁省农村信用社联合社',	'天津市商业银行',	'渤海银行',	'珠海华润银行股份有限公司',	'桂林银行',	'四川省农村信用社联合社',	'海南省农村信用社联合社',	'四川省农村信用社联合社',	'遵义市商业银行',	'苏州银行股份有限公司',	'广发银行股份有限公司',	'广州银行股份有限公司',	'中国工商银行卡拉奇分行',	'重庆农村商业银行',	'建设银行',	'建设银行',	'北京银行',	'广东华兴银行',	'泰隆城市信用社',	'晋中市榆次融信村镇银行',	'成都商业银行',	'招商银行',	'BC卡公司',	'招商银行',	'招商银行',	'中国建设银行亚洲股份有限公司',	'建设银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'上海农商银行',	'兰州银行股份有限公司',	'曲靖市商业银行',	'乐山市商业银行',	'建设银行',	'工商银行',	'工商银行',	'朝阳银行',	'成都农村商业银行',	'青海省农村信用社联合社',	'广东省农村信用社联合社',	'湖南省农村信用社联合社',	'甘肃省农村信用社联合社',	'山东省农村信用社联合社',	'广州农村商业银行',	'湖北农信',	'新疆农村信用社联合社',	'安徽省农村信用社联合社',	'浙江省农村信用社联合社',	'重庆农村商业银行',	'宁夏银行',	'四川省农村信用社联合社',	'吉林农信联合社',	'成都商业银行',	'内蒙古自治区农村信用社联合式',	'海南省农村信用社联合社',	'兰州银行',	'湖南省农村信用社联合社',	'黑龙江省农村信用社联合社',	'中国工商银行',	'中国工商银行',	'北京农村商业银行',	'黄河农村商业银行',	'桂林银行',	'中银通商务支付有限公司',	'陕西省农村信用社联合社',	'中国银行',	'中国银行',	'哈尔滨银行',	'江苏省农村信用社联合社',	'江苏银行',	'山西省农村信用社联合社',	'河南省农村信用社联合社',	'龙江银行',	'福建省农村信用社联合社',	'贵州省农村信用社联合社',	'安顺市商业银行',	'江西农信联合社',	'建设银行',	'邮政储蓄银行',	'厦门银行股份有限公司',	'濮阳银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行',	'农业银行',	'中国银行',	'建设银行',	'邮政储蓄银行',	'OJSCBASIAALLIANCEBANK',	'广东华兴银行',	'平安银行（深圳发展银行）',	'内蒙古银行',	'甘肃银行',	'中国银行胡志明分行',	'BC卡公司',	'Mongolia Trade Develop. Bank',	'巴基斯坦FAYSAL BANK',	'俄罗斯ORIENT EXPRESS BANK',	'中国银行金边分行',	'CJSC Fononbank',	'企业银行（中国）',	'Krung Thaj Bank Public Co. Ltd',	'宁波通商银行',	'巴基斯坦HabibBank',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国工商银行',	'农业银行',	'中国银行',	'中国建设银行',	'邮政储蓄银行',	'上海农商银行',	'山东省农村信用社联合社',	'民生银行',	'Capital Bank of Mongolia',	'本溪市商业银行',	'建设银行',	'福建省农村信用社联合社',	'中国工商银行新加坡分行',	'中国工商银行法兰克福分行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行印尼分行',	'中国银行',	'浙江民泰商业银行',	'广东南粤银行股份有限公司',	'广东省农村信用社联合社',	'工行东京分行',	'中国工商银行伦敦子行',	'中国工商银行（澳门）',	'中国工商银行（澳门）',	'工行首尔',	'贵阳银行股份有限公司',	'阳泉市商业银行',	'长沙银行股份有限公司',	'渣打银行（香港）',	'中国银行（香港）',	'集友银行',	'南洋商业银行',	'星展银行香港有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'商丘市商业银行',	'中国工商银行加拿大分行',	'中国工商银行马德里分行',	'乌鲁木齐市商业银行',	'哈尔滨银行',	'信阳银行',	'乌鲁木齐市商业银行',	'重庆三峡银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'包商银行',	'中国工商银行加拿大分行',	'中国工商银行（印度尼西亚）',	'工银河内',	'工银马来西亚',	'中国工商银行新加坡分行',	'昆仑银行股份有限公司',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'徽商银行',	'南京银行',	'安徽省农村信用社',	'晋中银行',	'中国工商银行伦敦子行',	'中国银行（香港）',	'南洋商业银行',	'集友银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'邮政储蓄银行',	'邮政储蓄银行',	'邮政储蓄银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'温州银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'阜新银行股份有限公司',	'福建省农村信用社联合社',	'深圳农村商业银行',	'深圳农村商业银行',	'日本三菱信用卡公司',	'江苏银行',	'嘉兴银行',	'贵阳银行股份有限公司',	'重庆银行',	'成都商业银行',	'西安银行股份有限公司',	'徽商银行芜湖分行',	'北京农村商业银行',	'兰州银行股份有限公司',	'廊坊银行',	'泰隆城市信用社',	'乌鲁木齐市商业银行',	'Allied Bank',	'青岛银行',	'内蒙古银行',	'上海银行',	'上海银行',	'邮储银行',	'邮储银行',	'成都商业银行',	'成都商业银行',	'成都商业银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国工商银行新加坡分行',	'中国工商银行法兰克福分行',	'中国光大银行',	'九江银行股份有限公司',	'北京银行',	'The Bancorp Bank',	'包商银行',	'中国建设银行',	'唐山市商业银行',	'中国建设银行',	'湖南省农村信用社联合社',	'贵阳银行股份有限公司',	'中国工商银行(印尼)',	'上海银行',	'江苏银行',	'浦东发展银行',	'浦东发展银行',	'吉林银行',	'吉林银行',	'邮储银行',	'福建省农村信用社联合社',	'邮储银行',	'邮储银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'重庆农村商业银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行澳门分行',	'浦东发展银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'东亚银行(中国)有限公司',	'东亚银行(中国)有限公司',	'上海银行',	'上海银行',	'上海银行',	'龙江银行股份有限公司',	'广东南海农村商业银行',	'创兴银行有限公司',	'中国银行澳门分行',	'中国银行澳门分行',	'南昌银行',	'浦东发展银行',	'浦东发展银行',	'上海银行',	'上海银行',	'建设银行',	'宁波银行',	'宁波银行',	'江苏银行',	'交通银行',	'杭州市商业银行',	'浙江泰隆商业银行',	'浙江省农村信用社联合社',	'宁波鄞州农村合作银行',	'福建省农村信用社联合社',	'柳州银行股份有限公司',	'柳州银行股份有限公司',	'大新银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'平安银行（深圳发展银行）',	'上海银行',	'湖州市商业银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'青海银行',	'齐商银行',	'徐州市郊农村信用合作联社',	'工商银行',	'工商银行',	'工商银行',	'宁波银行',	'工商银行',	'宁波银行',	'山东农村信用联合社',	'山东省农村信用社联合社',	'台州银行',	'佛山顺德农村商业银行',	'常熟农村商业银行',	'江苏农信社',	'汉口银行',	'徽商银行马鞍山分行',	'东莞农村商业银行',	'天津农村商业银行',	'天津市商业银行',	'张家港农村商业银行',	'东莞市商业银行',	'南宁市商业银行',	'包商银行',	'连云港市商业银行',	'焦作市商业银行',	'鄞州农村合作银行',	'徽商银行淮北分行',	'江阴农村商业银行',	'攀枝花市商业银行',	'佛山市三水区农村信用合作社',	'成都农村商业银行股份有限公司',	'中国银行香港有限公司',	'中国银行香港有限公司',	'中国银行香港有限公司',	'南洋商业银行',	'南洋商业银行',	'南洋商业银行',	'集友银行',	'集友银行',	'Bangkok Bank Pcl',	'集友银行',	'CSC',	'河北省农村信用社联合社',	'临沂商业银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'广东省农村信用社联合社',	'珠海华润银行股份有限公司',	'东亚银行有限公司',	'徽商银行安庆分行',	'绵阳市商业银行',	'长沙银行股份有限公司',	'昆明农联社',	'泉州银行',	'花旗银行有限公司',	'东亚银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'恒生银行有限公司',	'恒生银行有限公司',	'恒生银行有限公司',	'齐鲁银行股份有限公司',	'中国建设银行亚洲股份有限公司',	'中国建设银行亚洲股份有限公司',	'大连银行',	'恒丰银行',	'大连银行',	'上海商业银行',	'永隆银行有限公司',	'福建海峡银行股份有限公司',	'宁波鄞州农村合作银行',	'潍坊银行',	'泸州市商业银行',	'厦门银行股份有限公司',	'镇江市商业银行',	'大同市商业银行',	'湖北银行',	'宜昌市商业银行',	'葫芦岛市商业银行',	'辽阳银行股份有限公司',	'营口银行股份有限公司',	'工商银行',	'工商银行',	'工商银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'恒生银行',	'恒生银行',	'威海商业银行',	'湖北农信社',	'鞍山银行',	'蒙古郭勒姆特银行',	'丹东银行股份有限公司',	'南通商业银行',	'洛阳银行',	'郑州银行股份有限公司',	'江苏银行',	'永隆银行有限公司',	'哈尔滨银行',	'天津市商业银行',	'台州银行',	'宁夏银行',	'宁夏银行',	'大西洋银行股份有限公司',	'新加坡大华银行',	'澳门国际银行',	'澳门国际银行',	'澳门国际银行',	'广州农村商业银行股份有限公司',	'吉林银行',	'三门峡银行',	'抚顺银行股份有限公司',	'昆山农信社',	'蒙古郭勒姆特银行',	'江苏银行',	'广东南粤银行',	'金华银行股份有限公司',	'金华银行股份有限公司',	'大新银行有限公司',	'江苏农信社',	'中信嘉华银行有限公司',	'盛京银行',	'中信嘉华银行有限公司',	'中信嘉华银行有限公司',	'常熟市农村商业银行',	'创兴银行有限公司',	'晋商银行',	'盛京银行',	'广州银行股份有限公司',	'上海银行',	'云南省农村信用社',	'广东省农村信用社联合社',	'东亚银行有限公司',	'东亚银行有限公司',	'龙江银行股份有限公司',	'乌鲁木齐市商业银行',	'广东省农村信用社联合社',	'上海农商银行',	'中国银行澳门分行',	'中国银行澳门分行',	'江阴市农村商业银行',	'渣打银行香港有限公司',	'渣打银行香港有限公司',	'渣打银行香港有限公司',	'无锡市商业银行',	'绍兴银行股份有限公司',	'星展银行香港有限公司',	'吴江农商行',	'大新银行有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'AEON信贷财务亚洲有限公司',	'Travelex',	'Travelex',	'河北银行股份有限公司',	'河北银行股份有限公司',	'浦东发展银行',	'工商银行',	'工商银行',	'工商银行',	'陕西省农村信用社联合社',	'六盘水商行',	'工商银行',	'工商银行',	'湖北银行',	'工商银行',	'浦东发展银行',	'工商银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'天津农村商业银行',	'晋城银行股份有限公司',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'大丰银行有限公司',	'大丰银行有限公司',	'大丰银行有限公司',	'哈萨克斯坦国民储蓄银行',	'哈萨克斯坦国民储蓄银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'德阳银行',	'德阳银行',	'德阳银行',	'浙江泰隆商业银行',	'汉口银行',	'汉口银行',	'广发银行股份有限公司',	'无锡农村商业银行',	'中国光大银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'南京银行',	'南京银行',	'中国工商银行',	'招商银行',	'中国工商银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'工商银行',	'工商银行',	'工商银行',	'招商银行',	'上海农商银行贷记卡',	'重庆银行股份有限公司',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'汉口银行',	'汉口银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'龙江银行',	'吴江农商行',	'中国光大银行',	'徽商银行',	'中国光大银行',	'交通银行',	'光大银行',	'中国光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'中国建设银行亚洲股份有限公司',	'中国建设银行亚洲股份有限公司',	'中国建设银行亚洲股份有限公司',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'江西农信联合社',	'江西农信联合社',	'江西农信联合社',	'渤海银行',	'光大银行',	'浙江省农村信用社联合社',	'中国光大银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'建设银行',	'工商银行',	'工商银行',	'建设银行',	'中国建设银行',	'工商银行',	'浙江泰隆商业银行',	'上海农商银行贷记卡',	'建设银行',	'建设银行',	'浙江民泰商业银行',	'中国银行澳门分行',	'中国银行澳门分行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行澳门分行',	'中国银行澳门分行',	'中国银行澳门分行',	'澳门永亨银行股份有限公司',	'曲靖市商业银行',	'宁波银行',	'澳门永亨银行股份有限公司',	'中国银行',	'中行新加坡分行',	'中行新加坡分行',	'永亨银行',	'中国光大银行',	'福建省农村信用社联合社',	'工商银行',	'成都农村商业银行股份有限公司',	'哈尔滨商行',	'中国邮政储蓄银行信用卡中心',	'中国邮政储蓄银行信用卡中心',	'中国邮政储蓄银行信用卡中心',	'成都市商业银行',	'江苏省农村信用社联合社',	'江苏省农村信用社联合社',	'青海银行',	'成都市商业银行',	'中国农业银行贷记卡',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'天津农村商业银行',	'中国农业银行贷记卡',	'农业银行',	'农业银行',	'中国农业银行贷记卡',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'北京银行',	'北京银行',	'北京银行',	'苏州银行股份有限公司',	'桂林市商业银行',	'日照市商业银行',	'浙江省农村信用社联合社',	'珠海农村商业银行',	'龙江银行股份有限公司',	'澳门永亨银行股份有限公司',	'澳门永亨银行股份有限公司',	'莱芜银行',	'吉林银行',	'徐州市商业银行',	'重庆农村商业银行股份有限公司',	'温州银行',	'太仓农村商业银行',	'江苏长江商业银行',	'永亨银行',	'江苏银行',	'江苏银行',	'徽商银行',	'杭州商业银行',	'徽商银行',	'柳州银行股份有限公司',	'柳州银行股份有限公司',	'尧都区农村信用合作社联社',	'渤海银行',	'重庆农村商业银行股份有限公司',	'烟台商业银行',	'东莞商行',	'中国工商银行(亚洲)有限公司',	'江南农村商业银行',	'上海银行',	'贵州省农村信用社联合社',	'无锡农村商业银行',	'南充市商业银行',	'长沙银行股份有限公司',	'温州银行',	'长沙银行股份有限公司',	'兴业银行',	'工商银行',	'工商银行',	'工商银行',	'湖南农村信用社联合社',	'兴业银行',	'兴业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'日本三井住友卡公司',	'河北银行股份有限公司',	'兴业银行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'澳门永亨银行股份有限公司',	'东亚银行中国有限公司',	'吉林农信联合社',	'承德市商业银行',	'德州银行',	'东亚银行中国有限公司',	'遵义市商业银行',	'石嘴山银行',	'AEON信贷财务亚洲有限公司',	'渣打银行中国有限公司',	'东亚银行有限公司',	'中国工商银行(亚洲)有限公司',	'重庆三峡银行',	'汇丰银(中国)有限公司',	'黄河农村商业银行',	'渣打银行香港有限公司',	'中国工商银行(亚洲)有限公司',	'恒生银行',	'恒生银行',	'上海商业银行',	'安徽省农村信用社联合社',	'可汗银行',	'盛京银行',	'永亨银行',	'永亨银行',	'湖北银行',	'邯郸市商业银行',	'安顺市商业银行',	'长治市商业银行',	'永亨银行',	'建设银行',	'赣州银行股份有限公司',	'山西省农村信用社联合社',	'永隆银行有限公司',	'永隆银行有限公司',	'泰安市商业银行',	'乌海银行股份有限公司',	'AEON信贷财务亚洲有限公司',	'内蒙古自治区农村信用社联合式',	'龙江银行股份有限公司',	'鄂尔多斯银行股份有限公司',	'鹤壁银行',	'玉溪市商业银行',	'西安银行股份有限公司',	'张家口市商业银行股份有限公司',	'平安银行（深圳发展银行）（借记卡）',	'上海银行',	'平安银行（深圳发展银行）',	'上海银行',	'建设银行',	'平安银行（深圳发展银行）',	'锦州银行股份有限公司',	'河南省农村信用社联合社',	'广西农村信用社联合社',	'大连银行',	'渣打银行中国有限公司',	'湖北嘉鱼吴江村镇银行',	'成都商业银行',	'成都商业银行',	'中信银行',	'中信银行',	'河北银行股份有限公司',	'广西北部湾银行',	'工商银行',	'秦皇岛银行股份有限公司',	'工商银行',	'临商银行',	'工商银行',	'东莞市商业银行',	'工商银行',	'工商银行',	'河南省农村信用社联合社',	'工商银行',	'工商银行',	'重庆银行',	'青海省农村信用社联合社',	'农业银行',	'厦门银行股份有限公司',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'新韩卡公司',	'河北省农村信用社联合社',	'绍兴银行',	'陕西省农村信用社联合社',	'陕西省农村信用社联合社',	'汉口银行',	'朝阳银行',	'东亚银行中国有限公司',	'龙江银行',	'武汉农村商业银行',	'工银新西兰',	'深圳农村商业银行',	'福建省农村信用社联合社',	'漯河银行',	'东莞农村商业银行',	'台州银行',	'中国银行（香港）',	'南洋商业银行',	'集友银行',	'大西洋银行股份有限公司',	'攀枝花市商业银行',	'山西省农村信用社联合社',	'湖北农信社',	'平安银行（深圳发展银行）',	'河南省农村信用社联合社',	'承德银行',	'杭州商业银行',	'中国工商银行',	'福建海峡银行股份有限公司',	'大西洋银行股份有限公司',	'甘肃省农村信用社联合社',	'江苏省农村信用社联合社',	'金华银行股份有限公司',	'衡水银行',	'大连银行',	'大连银行',	'常熟农村商业银行',	'南充市商业银行',	'廊坊银行',	'太仓农村商业银行',	'湖北银行',	'日照银行',	'恒丰银行',	'宜宾市商业银行',	'盛京银行',	'浙江稠州商业银行',	'雅安市商业银行',	'泸州市商业银行',	'绍兴银行',	'枣庄银行',	'成都农村商业银行股份有限公司',	'宁夏银行',	'湖南省农村信用社联合社',	'浙江省农村信用社联合社',	'江西省农村信用社联合社',	'鄂尔多斯银行股份有限公司',	'黄河农村商业银行',	'重庆银行',	'丹东银行',	'抚顺银行',	'工商银行',	'德州银行',	'威海市商业银行',	'天津农村商业银行',	'汉口银行',	'恒生银行',	'恒生银行',	'盛京银行',	'天津滨海农村商业银行',	'吴江农商行',	'北京银行',	'广东华兴银行',	'江阴农村商业银行',	'济宁银行股份有限公司',	'洛阳银行',	'齐商银行',	'石嘴山银行',	'自贡市商业银行',	'张家港农村商业银行',	'无锡农村商业银行',	'广东南粤银行股份有限公司',	'兰州银行股份有限公司',	'吉林银行',	'昆山农信社',	'广西农村信用社联合社',	'玉溪市商业银行',	'驻马店银行',	'昆仑银行股份有限公司',	'辽阳银行股份有限公司',	'平顶山银行股份有限公司',	'乌海银行股份有限公司',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'信阳银行',	'盘锦市商业银行',	'上海农商银行',	'西安银行股份有限公司',	'阜新银行',	'保定银行',	'青岛银行',	'湖州市商业银行',	'大连银行',	'大连银行',	'营口银行股份有限公司',	'遂宁是商业银行',	'吉林省农村信用社联合社',	'甘肃银行',	'上海银行',	'上海银行',	'深圳农村商业银行',	'鞍山银行',	'江南农村商业银行',	'昆明农联社',	'泰安市商业银行',	'凉山州商业银行',	'宁夏银行',	'四川省农村信用社联合社',	'工商银行',	'徽商银行',	'焦作市商业银行',	'农业银行',	'武汉农村商业银行',	'中国银行',	'杭州商业银行',	'中国建设银行',	'邮政储蓄银行',	'邮政储蓄银行',	'建设银行',	'Rawbank S.a.r.l',	'工商银行',	'PVB Card Corporation',	'PVB Card Corporation',	'PVB Card Corporation',	'U Microfinance Bank Limited',	'南洋商业银行',	'中国银行（香港）',	'集友银行',	'Ecobank Nigeria',	'Al Baraka Bank(Pakistan)',	'OJSC Hamkor bank',	'Fidelity Bank Plc',	'东亚银行有限公司',	'NongHyup Bank',	'PVB Card Corporation',	'中信嘉华银行有限公司',	'State Bank of Mauritius',	'K & R International Limited',	'中国工商银行(亚洲)有限公司',	'JSC ATFBank',	'JSC ATFBank',	'JSC ATFBank',	'OJSC Russian Investment Bank',	'NongHyup Bank',	'JSC Liberty Bank',	'State Bank of Mauritius',	'工商银行',	'工商银行',	'广发银行股份有限公司',	'海口联合农村商业银行',	'黑龙江省农村信用社联合社',	'龙江银行',	'苏州银行股份有限公司',	'邯郸市商业银行',	'工商银行',	'广东华兴银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中信嘉华银行有限公司',	'Cim Finance Ltd',	'NongHyup Bank',	'NongHyup Bank',	'Cim Finance Ltd',	'JSC ATFBank',	'NongHyup Bank',	'NongHyup Bank',	'工商银行',	'台州银行',	'越南西贡商业银行',	'BC卡公司',	'Royal Bank Open Stock Company',	'BC卡公司',	'菲律宾RCBC',	'创兴银行有限公司',	'创兴银行有限公司',	'永亨银行',	'BC卡公司',	'BC卡公司',	'Allied Bank',	'澳门商业银行',	'澳门商业银行',	'中国工商银行（澳门）',	'中国工商银行（澳门）',	'中国工商银行（澳门）',	'BC卡公司',	'中国工商银行法兰克福分行',	'中国工商银行法兰克福分行',	'BC卡公司',	'恒生银行有限公司',	'BC卡公司',	'恒生银行有限公司',	'BC卡公司',	'交通银行香港分行',	'交通银行香港分行',	'BC卡公司',	'BC卡公司',	'菲律宾BDO',	'香港上海汇丰银行有限公司',	'菲律宾BDO',	'浙江民泰商业银行',	'BC卡公司',	'中银信用卡(国际)有限公司',	'中银信用卡(国际)有限公司',	'南洋商业银行',	'南洋商业银行',	'集友银行',	'广州银行股份有限公司',	'集友银行',	'中银信用卡(国际)有限公司',	'南洋商业银行',	'集友银行',	'永隆银行有限公司',	'永隆银行有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'花旗中国',	'花旗中国',	'花旗中国',	'花旗中国',	'BC卡公司',	'BC卡公司',	'广州农村商业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'佛山顺德农村商业银行',	'龙江银行股份有限公司',	'花旗银行有限公司',	'恒生银行有限公司',	'南洋商业银行',	'南洋商业银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'上海银行',	'常熟市农村商业银行',	'BC卡公司',	'俄罗斯ORIENT EXPRESS BANK',	'BC卡公司',	'上海商业银行',	'工银新西兰',	'BC卡公司',	'BC卡公司',	'工行米兰',	'中国工商银行巴黎分行',	'中国工商银行(亚洲)有限公司',	'中国工商银行澳门分行',	'哈尔滨银行',	'BC卡公司',	'BC卡公司',	'越南Vietcombank',	'BC卡公司',	'BC卡公司',	'福建省农村信用社联合社',	'BC卡公司',	'BC卡公司',	'乌鲁木齐市商业银行',	'中国银行（香港）',	'BC卡公司',	'中国银行曼谷分行',	'中银信用卡(国际)有限公司',	'中银信用卡(国际)有限公司',	'中国银行澳门分行',	'澳门大丰银行',	'越南Vietcombank',	'临汾市尧都区农村信用合作联社',	'BC卡公司',	'BC卡公司',	'北京农商行',	'大丰银行有限公司',	'大丰银行有限公司',	'BC卡公司',	'BC卡公司',	'BC卡公司',	'东莞农村商业银行',	'BC卡公司',	'中国工商银行',	'中国工商银行',	'中国工商银行',	'中国银行',	'中国农业银行贷记卡',	'中国银行',	'中国银行',	'中国光大银行',	'上海银行',	'上海银行',	'上海银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国建设银行',	'中国建设银行',	'上海农商银行贷记卡',	'乌鲁木齐市商业银行',	'乌鲁木齐市商业银行',	'湖南农信',	'成都农村商业银行',	'湖南农信',	'北京农商行',	'宁夏银行',	'中国银行',	'哈尔滨银行',	'龙江银行',	'河北银行股份有限公司',	'徽商银行',	'徽商银行',	'中国工商银行',	'中国工商银行',	'杭州市商业银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'韩国KB',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'韩国三星卡公司',	'韩国三星卡公司',	'天津农村商业银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国农业银行贷记卡',	'中国农业银行贷记卡',	'徽商银行',	'Bank of China(Malaysia)',	'浦发银行信用卡中心',	'广州银行股份有限公司',	'上海银行',	'新韩卡公司',	'新韩卡公司',	'Kasikorn Bank PCL',	'中国工商银行',	'中国工商银行',	'东莞农村商业银行',	'中国工商银行',	'中国工商银行(亚洲)有限公司',	'长沙银行股份有限公司',	'江苏银行',	'宁波银行',	'韩国乐天',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'民生银行',	'民生银行',	'民生银行',	'中国工商银行阿拉木图子行',	'中国工商银行(亚洲)有限公司',	'中国工商银行(亚洲)有限公司',	'中国工商银行新加坡分行',	'中国工商银行印尼分行',	'中国邮政储蓄银行信用卡中心',	'工银泰国',	'工行马来西亚',	'工行河内',	'工行伦敦',	'中国工商银行卢森堡分行',	'工行阿姆斯特丹',	'中国工商银行（巴西）',	'工行阿拉木图',	'工行布鲁塞尔',	'工行加拿大',	'工银印尼',	'工银法兰克福',	'工行马德里',	'工银河内',	'中国工商银行金边分行',	'大新银行有限公司',	'中国工商银行卡拉奇分行',	'Bank of China(Malaysia)',	'汉口银行',	'包商银行',	'哈尔滨银行',	'上海银行',	'建设银行',	'建设银行',	'浦东发展银行',	'浦东发展银行',	'青海银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'中国建设银行',	'中国建设银行',	'中国建设银行',	'华夏银行',	'华夏银行',	'华夏银行',	'浦发银行信用卡中心',	'浦发银行信用卡中心',	'东亚银行(中国)有限公司',	'东亚银行(中国)有限公司',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国工商银行万象分行',	'中国工商银行金边分行',	'温州银行',	'上海农商银行贷记卡',	'浦东发展银行',	'天津市商业银行',	'农业银行',	'农业银行',	'农业银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'汉口银行',	'中国光大银行',	'中国光大银行',	'北京银行',	'金华银行股份有限公司',	'天津银行',	'中信银行信用卡中心',	'宁波银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'江苏银行',	'上海农商银行贷记卡',	'兴业银行',	'贵阳银行股份有限公司',	'宁夏银行',	'交通银行',	'东营银行',	'交通银行',	'桂林银行',	'重庆银行股份有限公司',	'浦东发展银行',	'浦东发展银行',	'上饶银行',	'哈尔滨商行',	'江苏省农村信用社联合社',	'广西北部湾银行',	'成都市商业银行',	'承德银行',	'上海银行',	'富滇银行',	'福建省农村信用社联合社',	'赣州银行股份有限公司',	'威海市商业银行',	'广州农村商业银行',	'杭州市商业银行',	'广东南粤银行',	'九江银行股份有限公司',	'青岛银行',	'南京银行',	'吴江农商行',	'浙江稠州商业银行',	'徽商银行',	'内蒙古银行',	'鄂尔多斯银行',	'温州银行',	'日照银行',	'民生银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'锦州银行',	'招商银行信用卡中心',	'兰州银行',	'中国建设银行',	'重庆农村商业银行',	'农业银行',	'中国农业银行贷记卡',	'珠海华润银行',	'商丘市商业银行',	'常熟市农村商业银行',	'台州银行',	'浙江泰隆商业银行',	'乌鲁木齐市商业银行',	'莱商银行',	'浙江省农村信用社联合社',	'长沙银行股份有限公司',	'长沙银行股份有限公司',	'长沙银行股份有限公司',	'盛京银行',	'工商银行',	'青海银行',	'工商银行',	'青海省农村信用社联合社',	'招商银行',	'绍兴银行',	'柳州银行',	'包商银行',	'平安银行（深圳发展银行）股份有限公司',	'四川省农村信用社联合社',	'大连银行',	'宁波鄞州农村合作银行',	'天津农村商业银行',	'漯河银行',	'南昌银行',	'湖州银行',	'云南省农村信用社',	'成都农村商业银行股份有限公司',	'辽阳银行股份有限公司',	'中国邮政储蓄银行信用卡中心',	'齐商银行',	'中国银行',	'中国银行',	'周口银行',	'中国建设银行',	'中国建设银行',	'华夏银行',	'泉州银行',	'河北银行股份有限公司',	'佛山顺德农村商业银行',	'陕西省农村信用社联合社',	'华融湘江银行',	'丹东银行',	'广西壮族自治区农村信用社联合社',	'信阳银行',	'甘肃省农村信用社联合社',	'龙江银行股份有限公司',	'北京农商行',	'平顶山银行',	'玉溪市商业银行',	'石嘴山银行',	'吉林银行',	'临商银行',	'招商银行信用卡中心',	'中国建设银行',	'广州银行股份有限公司',	'龙江银行股份有限公司',	'无锡农村商业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'鄂尔多斯银行股份有限公司',	'齐鲁银行股份有限公司',	'江阴农村商业银行',	'临汾市尧都区农村信用合作联社',	'湖南农村信用社联合社',	'中国银行',	'驻马店银行',	'潍坊银行',	'江西省农村信用社联合社',	'德州银行',	'东莞商行',	'泉州银行',	'广发银行股份有限公司',	'招商银行',	'长沙银行股份有限公司',	'工商银行',	'工商银行',	'山东农村信用联合社',	'山东农村信用联合社',	'广州农村商业银行股份有限公司',	'中银通商务支付有限公司',	'中银通支付',	'中银通商务支付有限公司',	'中银通商务支付有限公司',	'乌鲁木齐市商业银行',	'鞍山银行',	'锦州银行股份有限公司',	'南昌银行',	'齐鲁银行股份有限公司',	'吉林银行',	'上海农商银行',	'天津市商业银行',	'广州银行股份有限公司',	'南通商业银行',	'重庆银行',	'重庆农村商业银行股份有限公司',	'上海银行',	'宁波银行',	'厦门银行股份有限公司',	'镇江市商业银行',	'成都商业银行',	'昆山农信社',	'晋商银行',	'焦作市商业银行',	'恒丰银行',	'广州农村商业银行股份有限公司',	'广东南海农村商业银行',	'佛山顺德农村商业银行',	'盛京银行',	'兰州银行股份有限公司',	'洛阳银行',	'昆明农联社',	'辽阳银行股份有限公司',	'武汉农信',	'河北银行股份有限公司',	'吉林银行',	'珠海华润银行股份有限公司',	'哈尔滨银行',	'东莞市商业银行',	'金华银行股份有限公司',	'抚顺银行股份有限公司',	'葫芦岛市商业银行',	'宜昌市商业银行',	'郑州银行股份有限公司',	'齐商银行',	'江阴农村商业银行',	'丹东银行股份有限公司',	'青岛银行',	'东莞农村商业银行',	'张家港农村商业银行',	'日照市商业银行',	'临沂商业银行',	'青海银行',	'台州银行',	'盐城商行',	'长沙银行股份有限公司',	'潍坊银行',	'赣州银行股份有限公司',	'泉州银行',	'江苏银行',	'邮储银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'兴业银行',	'中信银行',	'中信银行',	'中信银行',	'浦东发展银行',	'浦东发展银行',	'佛山顺德农村商业银行',	'汉口银行',	'昆明商业银行',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'华夏银行'],\
                    }
    list_yinhang = ['广东发展银行',	'农业银行',	'昆明农联社',	'中国光大银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中信实业银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'华夏',	'中国银行',	'工商银行',	'农业银行',	'交通银行',	'广东发展银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'工商银行',	'建设银行',	'建设银行',	'农业银行',	'农业银行',	'交通银行',	'中国银行',	'中国银行',	'中国银行',	'广东发展银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'深圳农联社',	'南京市商业银行',	'农业银行',	'上海银行',	'南通商业银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'贵阳市商业银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'广东发展银行',	'上海浦东发展银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'上海银行',	'上海银行',	'上海银行',	'上海银行',	'中国银行',	'中国银行',	'光大银行',	'光大银行',	'中国光大银行',	'中国光大银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国工商银行',	'中国工商银行',	'中国工商银行',	'中国工商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'中国建设银行',	'中国建设银行',	'中国建设银行',	'中国工商银行',	'中国工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'招商银行信用卡中心',	'招商银行信用卡中心',	'中国工商银行',	'中国工商银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'浦发银行信用卡中心',	'中国银行',	'中信银行信用卡中心',	'招商银行',	'上海银行',	'上海银行',	'工商银行',	'农业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'浦东发展银行',	'浦东发展银行',	'交通银行',	'中国光大银行',	'中国光大银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'民生银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'招商银行',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'民生银行',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'北京银行',	'建设银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'北京银行',	'北京银行',	'中国光大银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'民生银行',	'民生银行',	'广发银行股份有限公司',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行',	'中信银行',	'中信银行',	'建设银行',	'建设银行',	'交通银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国建设银行',	'建设银行',	'中国建设银行',	'建设银行',	'中国建设银行',	'中国建设银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'中国银行',	'工商银行',	'工商银行',	'兴业银行',	'兴业银行',	'上海银行',	'招商银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'招商银行',	'中信银行',	'中信银行',	'兴业银行',	'兴业银行',	'工商银行',	'工商银行',	'工商银行',	'建设银行',	'中国银行',	'浦东发展银行',	'工商银行',	'交通银行',	'交通银行',	'工商银行',	'兴业银行',	'农业银行',	'民生银行',	'民生银行',	'招商银行',	'民生银行',	'民生银行',	'招商银行',	'招商银行',	'中国光大银行',	'平安银行（深圳发展银行）股份有限公司',	'上海银行',	'兴业银行',	'兴业银行',	'光大银行',	'兴业银行',	'广发银行股份有限公司',	'中国建设银行',	'工商银行',	'工商银行',	'工商银行',	'建设银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'浦东发展银行',	'江苏银行',	'工商银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'招商银行',	'宁波银行',	'民生银行',	'工商银行',	'农业银行',	'中信银行信用卡中心',	'中国银行',	'中国银行',	'浦东发展银行',	'民生银行',	'浦东发展银行',	'中信银行信用卡中心',	'广发银行股份有限公司',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'农业银行',	'农业银行',	'上海银行',	'上海银行',	'农业银行',	'农业银行',	'中信银行信用卡中心',	'上海银行',	'广发银行股份有限公司',	'交通银行',	'宁波银行',	'广发银行股份有限公司',	'招商银行',	'交通银行',	'北京银行',	'交通银行',	'兴业银行',	'民生银行',	'华夏银行',	'招商银行',	'上海银行',	'工商银行',	'兴业银行',	'中国光大银行',	'工商银行',	'建设银行',	'中国工商银行',	'中国银行',	'中国银行',	'工商银行',	'中国银行',	'中国银行',	'浦东发展银行',	'建设银行',	'工商银行',	'平安银行（深圳发展银行）股份有限公司',	'兴业银行',	'平安银行（深圳发展银行）股份有限公司',	'兴业银行',	'华夏银行',	'华夏银行',	'中国工商银行',	'广发银行股份有限公司',	'民生银行',	'工商银行',	'工商银行',	'平安银行（深圳发展银行）股份有限公司',	'中国建设银行',	'中国建设银行',	'中国建设银行',	'华夏银行',	'华夏银行',	'工商银行',	'中国光大银行',	'建设银行',	'中国工商银行',	'中国农业银行贷记卡',	'中国建设银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'中国银行',	'工商银行',	'中国农业银行贷记卡',	'兴业银行',	'上海银行',	'广发银行股份有限公司',	'中国工商银行',	'兴业银行',	'中国工商银行',	'建设银行',	'民生银行',	'兴业银行',	'招商银行',	'招商银行',	'农业银行',	'中国银行',	'广发银行股份有限公司',	'中国建设银行',	'交通银行',	'中国银行',	'民生银行',	'建设银行',	'民生银行',	'中信银行信用卡中心',	'中国建设银行',	'工商银行',	'农业银行',	'中国银行',	'中国银行',	'广发银行股份有限公司',	'中国建设银行',	'中信银行信用卡中心',	'中国建设银行',	'盛京银行',	'建设银行',	'中国银行',	'交通银行',	'平安银行（深圳发展银行）',	'北京银行',	'杭州商业银行',	'广州银行股份有限公司',	'苏州市商业银行',	'徽商银行合肥分行',	'绍兴银行股份有限公司',	'常熟农村商业银行',	'大连银行',	'越南西贡商业银行',	'平安银行（深圳发展银行）（借记卡）',	'BC卡公司',	'交通银行香港分行',	'新加坡星网电子付款私人有限公司',	'中国银行(马来西亚)',	'交通银行',	'Royal Bank Open Stock Company',	'中国银行（澳大利亚）',	'中国银行（澳大利亚）',	'BC卡公司',	'中国工商银行(亚洲)有限公司',	'BC卡公司',	'中国银行(马来西亚)',	'广发银行股份有限公司',	'中银通商务支付有限公司',	'BC卡公司',	'中国银行',	'福建海峡银行股份有限公司',	'工银河内',	'中银通商务支付有限公司',	'中国工商银行(亚洲)有限公司',	'中国工商银行布鲁塞尔分行',	'工商银行',	'农业银行',	'建设银行',	'中国银行',	'邮储银行',	'中国银盛',	'中国银盛',	'大丰银行有限公司',	'Commercial Bank of Dubai',	'中信银行',	'光大银行',	'中国工商银行',	'北京农村商业银行',	'Commercial Bank of Dubai',	'中国工商银行新加坡分行',	'中国工商银行(亚洲)有限公司',	'BC卡公司',	'The Bancorp Bank',	'BC卡公司',	'中国建设银行（亚洲）',	'Russian Standard Bank',	'中国工商银行法兰克福分行',	'柳州银行股份有限公司',	'BC卡公司',	'BC卡公司',	'中国工商银行澳门分行',	'BC卡公司',	'澳门通股份有限公司',	'The Mauritius Commercial Bank',	'BC卡公司',	'中国工商银行布鲁塞尔分行',	'中国工商银行米兰分行',	'中国工商银行法兰克福分行',	'中国工商银行马德里分行',	'中国工商银行米兰分行',	'Travelex',	'Travelex',	'中国工商银行澳门分行',	'中国工商银行加拿大分行',	'中国工商银行马德里分行',	'中国工商银行新加坡分行',	'中国工商银行阿拉木图子行',	'工商银行',	'中国银行（香港）',	'中国银行（香港）',	'大丰银行有限公司',	'大丰银行有限公司',	'集友银行',	'集友银行',	'南洋商业银行',	'南洋商业银行',	'中国银行澳门分行',	'中国银行澳门分行',	'BC卡公司',	'BC卡公司',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'宁波鄞州农村合作银行',	'宁波市农业银行',	'工商银行',	'工商银行',	'工商银行',	'中行宁波分行',	'中国银行',	'中银通商务支付有限公司',	'工商银行',	'浙江民泰商业银行',	'光大银行',	'长沙银行股份有限公司',	'招商银行',	'交通银行',	'上海银行',	'中信银行',	'邢台银行',	'邮政储蓄银行',	'浦东发展银行',	'中国银行澳门分行',	'澳门通股份有限公司',	'光大银行',	'澳门通股份有限公司',	'中银通商务支付有限公司',	'中国工商银行澳门分行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'BC卡公司',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'交通银行香港分行',	'BC卡公司',	'东营银行',	'上海银行',	'BC卡公司',	'新疆维吾尔自治区农村信用社联合',	'平凉市商业银行',	'BC卡公司',	'BC卡公司',	'湖北农信社',	'上饶市商业银行',	'星展银行',	'星展银行',	'昆明农联社',	'广东省农村信用社联合社',	'浙商银行',	'BC卡公司',	'河北省农村信用社联合社',	'BC卡公司',	'宁波鄞州农村合作银行',	'BC卡公司',	'辽宁省农村信用社联合社',	'BC卡公司',	'浙江稠州商业银行',	'昆仑银行股份有限公司',	'北京银行',	'BC卡公司',	'BC卡公司',	'四川省农村信用社联合社',	'上海商业银行',	'许昌银行',	'海南省农村信用社联合社',	'龙江银行',	'铁岭银行',	'BC卡公司',	'莫斯科人民储蓄银行',	'中国银行(香港)',	'南洋商业银行',	'集友银行',	'宝鸡商行',	'丝绸之路银行',	'蒙古郭勒姆特银行',	'上海银行',	'福建建瓯石狮村镇银行',	'越南西贡商业银行',	'凉山州商业银行',	'浙江省农村信用社联合社',	'友利银行(中国)有限公司',	'甘肃省农村信用社联合社',	'花旗银行(中国)有限公司',	'花旗银行(中国)有限公司',	'AEON信贷财务亚洲有限公司',	'青海省农村信用社联合社',	'北京农村商业银行',	'北京农村商业银行',	'北京农村商业银行',	'交通银行香港分行',	'自贡市商业银行',	'平顶山银行股份有限公司',	'新韩银行',	'天津滨海农村商业银行',	'朝阳银行',	'漯河银行',	'江苏银行',	'华侨银行（中国）',	'BC卡公司',	'建设银行',	'建设银行',	'建设银行',	'中国建设银行亚洲股份有限公司',	'中国建设银行澳门股份有限公司',	'乐山市商业银行',	'创兴银行有限公司',	'浙江民泰商业银行',	'盘锦市商业银行',	'遂宁市商业银行',	'保定银行',	'邮储银行',	'邮储银行',	'龙江银行',	'邮储银行',	'四川省农村信用社联合社',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'济宁银行股份有限公司',	'韩亚银行（中国）',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国银行',	'南洋商业银行（中国）',	'中银东京分行',	'晋商银行',	'湖北银行',	'BC卡公司',	'驻马店银行',	'华夏银行',	'晋中银行',	'可汗银行',	'工商银行',	'工商银行',	'工商银行',	'黑龙江省农村信用社联合社',	'重庆璧山工银村镇银行',	'浙江平湖工银村镇银行',	'中国银行马尼拉分行',	'大西洋银行股份有限公司',	'达州市商业银行',	'CSC',	'新乡市商业银行',	'秦皇岛银行股份有限公司',	'邢台银行股份有限公司',	'衡水银行',	'中国工商银行(印尼)',	'兰州银行股份有限公司',	'盛京银行',	'法国兴业银行（中国）',	'大西洋银行股份有限公司',	'中国银行雅加达分行',	'中国银行首尔分行',	'福建省农村信用社联合社',	'青岛银行',	'澳门商业银行',	'澳门商业银行',	'澳门商业银行',	'中国银行',	'Baiduri Bank Berhad',	'重庆农村商业银行',	'南京银行',	'库尔勒市商业银行',	'俄罗斯远东商业银行',	'沧州银行股份有限公司',	'福建海峡银行股份有限公司',	'渤海银行',	'南昌银行',	'湖北银行',	'周口银行',	'阳泉市商业银行',	'宜宾市商业银行',	'澳门BDA',	'大新银行（中国）',	'宁波银行',	'山西省农村信用社',	'工商银行',	'农业银行',	'中国银行',	'建设银行',	'邮储银行',	'招商银行',	'新疆维吾尔自治区农村信用社联合',	'工商银行',	'南洋商业银行（中国）',	'南洋商业银行（中国）',	'南洋商业银行（中国）',	'南洋商业银行（中国）',	'中国银行（澳大利亚）',	'中国银行（澳大利亚）',	'越南Vietcombank',	'长安银行',	'中国银行',	'永亨银行（中国）有限公司',	'招商银行香港分行',	'中国工商银行巴黎分行',	'格鲁吉亚 Invest Bank',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'三水珠江村镇银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'澳门BDA',	'雅安市商业银行',	'安阳银行',	'大华银行（中国）',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行曼谷分行',	'交通银行澳门分行',	'农业银行',	'商丘商行',	'新疆汇和银行',	'廊坊银行',	'廊坊银行',	'中国银行（澳大利亚）',	'中国银行（澳大利亚）',	'Royal Bank Open Stock Company',	'宁波东海银行',	'营口沿海银行',	'乌兹别克斯坦INFINBANK',	'浦东发展银行',	'浦东发展银行',	'BCEL',	'景德镇商业银行',	'哈密市商业银行',	'武汉农村商业银行',	'内蒙古自治区农村信用社联合式',	'江南农村商业银行',	'中国银行（澳大利亚）',	'华融湘江银行',	'工银泰国',	'工银河内',	'中国工商银行卢森堡分行',	'中国工商银行米兰分行',	'中国工商银行阿姆斯特丹',	'中国工商银行伦敦子行',	'中国工商银行阿拉木图子行',	'中国工商银行布鲁塞尔分行',	'中国工商银行马德里分行',	'中国工商银行巴黎分行',	'中国工商银行法兰克福分行',	'华融湘江银行',	'浦东发展银行',	'西藏银行',	'南阳村镇银行',	'中国银行（澳大利亚）',	'中国银行曼谷分行',	'东营莱商村镇银行',	'工商银行',	'莱商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'东亚银行澳门分行',	'柳州银行股份有限公司',	'张家口市商业银行',	'中国工商银行阿拉木图子行',	'昆明商业银行',	'上饶银行',	'宁夏银行',	'宁波银行',	'青岛银行',	'北京银行',	'沧州银行',	'中国工商银行（巴西）',	'中国工商银行金边分行',	'中国工商银行万象分行',	'中国工商银行金边分行',	'交通银行香港分行',	'泉州银行',	'东莞市商业银行',	'恒生银行',	'恒生银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'长沙银行股份有限公司',	'长安银行',	'辽宁省农村信用社联合社',	'天津市商业银行',	'渤海银行',	'珠海华润银行股份有限公司',	'桂林银行',	'四川省农村信用社联合社',	'海南省农村信用社联合社',	'四川省农村信用社联合社',	'遵义市商业银行',	'苏州银行股份有限公司',	'广发银行股份有限公司',	'广州银行股份有限公司',	'中国工商银行卡拉奇分行',	'重庆农村商业银行',	'建设银行',	'建设银行',	'北京银行',	'广东华兴银行',	'泰隆城市信用社',	'晋中市榆次融信村镇银行',	'成都商业银行',	'招商银行',	'BC卡公司',	'招商银行',	'招商银行',	'中国建设银行亚洲股份有限公司',	'建设银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'上海农商银行',	'兰州银行股份有限公司',	'曲靖市商业银行',	'乐山市商业银行',	'建设银行',	'工商银行',	'工商银行',	'朝阳银行',	'成都农村商业银行',	'青海省农村信用社联合社',	'广东省农村信用社联合社',	'湖南省农村信用社联合社',	'甘肃省农村信用社联合社',	'山东省农村信用社联合社',	'广州农村商业银行',	'湖北农信',	'新疆农村信用社联合社',	'安徽省农村信用社联合社',	'浙江省农村信用社联合社',	'重庆农村商业银行',	'宁夏银行',	'四川省农村信用社联合社',	'吉林农信联合社',	'成都商业银行',	'内蒙古自治区农村信用社联合式',	'海南省农村信用社联合社',	'兰州银行',	'湖南省农村信用社联合社',	'黑龙江省农村信用社联合社',	'中国工商银行',	'中国工商银行',	'北京农村商业银行',	'黄河农村商业银行',	'桂林银行',	'中银通商务支付有限公司',	'陕西省农村信用社联合社',	'中国银行',	'中国银行',	'哈尔滨银行',	'江苏省农村信用社联合社',	'江苏银行',	'山西省农村信用社联合社',	'河南省农村信用社联合社',	'龙江银行',	'福建省农村信用社联合社',	'贵州省农村信用社联合社',	'安顺市商业银行',	'江西农信联合社',	'建设银行',	'邮政储蓄银行',	'厦门银行股份有限公司',	'濮阳银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行',	'农业银行',	'中国银行',	'建设银行',	'邮政储蓄银行',	'OJSCBASIAALLIANCEBANK',	'广东华兴银行',	'平安银行（深圳发展银行）',	'内蒙古银行',	'甘肃银行',	'中国银行胡志明分行',	'BC卡公司',	'Mongolia Trade Develop. Bank',	'巴基斯坦FAYSAL BANK',	'俄罗斯ORIENT EXPRESS BANK',	'中国银行金边分行',	'CJSC Fononbank',	'企业银行（中国）',	'Krung Thaj Bank Public Co. Ltd',	'宁波通商银行',	'巴基斯坦HabibBank',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国工商银行',	'农业银行',	'中国银行',	'中国建设银行',	'邮政储蓄银行',	'上海农商银行',	'山东省农村信用社联合社',	'民生银行',	'Capital Bank of Mongolia',	'本溪市商业银行',	'建设银行',	'福建省农村信用社联合社',	'中国工商银行新加坡分行',	'中国工商银行法兰克福分行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行印尼分行',	'中国银行',	'浙江民泰商业银行',	'广东南粤银行股份有限公司',	'广东省农村信用社联合社',	'工行东京分行',	'中国工商银行伦敦子行',	'中国工商银行（澳门）',	'中国工商银行（澳门）',	'工行首尔',	'贵阳银行股份有限公司',	'阳泉市商业银行',	'长沙银行股份有限公司',	'渣打银行（香港）',	'中国银行（香港）',	'集友银行',	'南洋商业银行',	'星展银行香港有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'商丘市商业银行',	'中国工商银行加拿大分行',	'中国工商银行马德里分行',	'乌鲁木齐市商业银行',	'哈尔滨银行',	'信阳银行',	'乌鲁木齐市商业银行',	'重庆三峡银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'包商银行',	'中国工商银行加拿大分行',	'中国工商银行（印度尼西亚）',	'工银河内',	'工银马来西亚',	'中国工商银行新加坡分行',	'昆仑银行股份有限公司',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'徽商银行',	'南京银行',	'安徽省农村信用社',	'晋中银行',	'中国工商银行伦敦子行',	'中国银行（香港）',	'南洋商业银行',	'集友银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'邮政储蓄银行',	'邮政储蓄银行',	'邮政储蓄银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'温州银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'阜新银行股份有限公司',	'福建省农村信用社联合社',	'深圳农村商业银行',	'深圳农村商业银行',	'日本三菱信用卡公司',	'江苏银行',	'嘉兴银行',	'贵阳银行股份有限公司',	'重庆银行',	'成都商业银行',	'西安银行股份有限公司',	'徽商银行芜湖分行',	'北京农村商业银行',	'兰州银行股份有限公司',	'廊坊银行',	'泰隆城市信用社',	'乌鲁木齐市商业银行',	'Allied Bank',	'青岛银行',	'内蒙古银行',	'上海银行',	'上海银行',	'邮储银行',	'邮储银行',	'成都商业银行',	'成都商业银行',	'成都商业银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国工商银行新加坡分行',	'中国工商银行法兰克福分行',	'中国光大银行',	'九江银行股份有限公司',	'北京银行',	'The Bancorp Bank',	'包商银行',	'中国建设银行',	'唐山市商业银行',	'中国建设银行',	'湖南省农村信用社联合社',	'贵阳银行股份有限公司',	'中国工商银行(印尼)',	'上海银行',	'江苏银行',	'浦东发展银行',	'浦东发展银行',	'吉林银行',	'吉林银行',	'邮储银行',	'福建省农村信用社联合社',	'邮储银行',	'邮储银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'重庆农村商业银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行澳门分行',	'浦东发展银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中国工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'东亚银行(中国)有限公司',	'东亚银行(中国)有限公司',	'上海银行',	'上海银行',	'上海银行',	'龙江银行股份有限公司',	'广东南海农村商业银行',	'创兴银行有限公司',	'中国银行澳门分行',	'中国银行澳门分行',	'南昌银行',	'浦东发展银行',	'浦东发展银行',	'上海银行',	'上海银行',	'建设银行',	'宁波银行',	'宁波银行',	'江苏银行',	'交通银行',	'杭州市商业银行',	'浙江泰隆商业银行',	'浙江省农村信用社联合社',	'宁波鄞州农村合作银行',	'福建省农村信用社联合社',	'柳州银行股份有限公司',	'柳州银行股份有限公司',	'大新银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'平安银行（深圳发展银行）',	'上海银行',	'湖州市商业银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'青海银行',	'齐商银行',	'徐州市郊农村信用合作联社',	'工商银行',	'工商银行',	'工商银行',	'宁波银行',	'工商银行',	'宁波银行',	'山东农村信用联合社',	'山东省农村信用社联合社',	'台州银行',	'佛山顺德农村商业银行',	'常熟农村商业银行',	'江苏农信社',	'汉口银行',	'徽商银行马鞍山分行',	'东莞农村商业银行',	'天津农村商业银行',	'天津市商业银行',	'张家港农村商业银行',	'东莞市商业银行',	'南宁市商业银行',	'包商银行',	'连云港市商业银行',	'焦作市商业银行',	'鄞州农村合作银行',	'徽商银行淮北分行',	'江阴农村商业银行',	'攀枝花市商业银行',	'佛山市三水区农村信用合作社',	'成都农村商业银行股份有限公司',	'中国银行香港有限公司',	'中国银行香港有限公司',	'中国银行香港有限公司',	'南洋商业银行',	'南洋商业银行',	'南洋商业银行',	'集友银行',	'集友银行',	'Bangkok Bank Pcl',	'集友银行',	'CSC',	'河北省农村信用社联合社',	'临沂商业银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'广东省农村信用社联合社',	'珠海华润银行股份有限公司',	'东亚银行有限公司',	'徽商银行安庆分行',	'绵阳市商业银行',	'长沙银行股份有限公司',	'昆明农联社',	'泉州银行',	'花旗银行有限公司',	'东亚银行有限公司',	'大新银行有限公司',	'大新银行有限公司',	'恒生银行有限公司',	'恒生银行有限公司',	'恒生银行有限公司',	'齐鲁银行股份有限公司',	'中国建设银行亚洲股份有限公司',	'中国建设银行亚洲股份有限公司',	'大连银行',	'恒丰银行',	'大连银行',	'上海商业银行',	'永隆银行有限公司',	'福建海峡银行股份有限公司',	'宁波鄞州农村合作银行',	'潍坊银行',	'泸州市商业银行',	'厦门银行股份有限公司',	'镇江市商业银行',	'大同市商业银行',	'湖北银行',	'宜昌市商业银行',	'葫芦岛市商业银行',	'辽阳银行股份有限公司',	'营口银行股份有限公司',	'工商银行',	'工商银行',	'工商银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'恒生银行',	'恒生银行',	'威海商业银行',	'湖北农信社',	'鞍山银行',	'蒙古郭勒姆特银行',	'丹东银行股份有限公司',	'南通商业银行',	'洛阳银行',	'郑州银行股份有限公司',	'江苏银行',	'永隆银行有限公司',	'哈尔滨银行',	'天津市商业银行',	'台州银行',	'宁夏银行',	'宁夏银行',	'大西洋银行股份有限公司',	'新加坡大华银行',	'澳门国际银行',	'澳门国际银行',	'澳门国际银行',	'广州农村商业银行股份有限公司',	'吉林银行',	'三门峡银行',	'抚顺银行股份有限公司',	'昆山农信社',	'蒙古郭勒姆特银行',	'江苏银行',	'广东南粤银行',	'金华银行股份有限公司',	'金华银行股份有限公司',	'大新银行有限公司',	'江苏农信社',	'中信嘉华银行有限公司',	'盛京银行',	'中信嘉华银行有限公司',	'中信嘉华银行有限公司',	'常熟市农村商业银行',	'创兴银行有限公司',	'晋商银行',	'盛京银行',	'广州银行股份有限公司',	'上海银行',	'云南省农村信用社',	'广东省农村信用社联合社',	'东亚银行有限公司',	'东亚银行有限公司',	'龙江银行股份有限公司',	'乌鲁木齐市商业银行',	'广东省农村信用社联合社',	'上海农商银行',	'中国银行澳门分行',	'中国银行澳门分行',	'江阴市农村商业银行',	'渣打银行香港有限公司',	'渣打银行香港有限公司',	'渣打银行香港有限公司',	'无锡市商业银行',	'绍兴银行股份有限公司',	'星展银行香港有限公司',	'吴江农商行',	'大新银行有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'星展银行香港有限公司',	'AEON信贷财务亚洲有限公司',	'Travelex',	'Travelex',	'河北银行股份有限公司',	'河北银行股份有限公司',	'浦东发展银行',	'工商银行',	'工商银行',	'工商银行',	'陕西省农村信用社联合社',	'六盘水商行',	'工商银行',	'工商银行',	'湖北银行',	'工商银行',	'浦东发展银行',	'工商银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'浦东发展银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'天津农村商业银行',	'晋城银行股份有限公司',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）（借记卡）',	'大丰银行有限公司',	'大丰银行有限公司',	'大丰银行有限公司',	'哈萨克斯坦国民储蓄银行',	'哈萨克斯坦国民储蓄银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'德阳银行',	'德阳银行',	'德阳银行',	'浙江泰隆商业银行',	'汉口银行',	'汉口银行',	'广发银行股份有限公司',	'无锡农村商业银行',	'中国光大银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'招商银行',	'南京银行',	'南京银行',	'中国工商银行',	'招商银行',	'中国工商银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'工商银行',	'工商银行',	'工商银行',	'招商银行',	'上海农商银行贷记卡',	'重庆银行股份有限公司',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'民生银行',	'汉口银行',	'汉口银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'龙江银行',	'吴江农商行',	'中国光大银行',	'徽商银行',	'中国光大银行',	'交通银行',	'光大银行',	'中国光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'中国建设银行亚洲股份有限公司',	'中国建设银行亚洲股份有限公司',	'中国建设银行亚洲股份有限公司',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'江西农信联合社',	'江西农信联合社',	'江西农信联合社',	'渤海银行',	'光大银行',	'浙江省农村信用社联合社',	'中国光大银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'中信银行',	'建设银行',	'工商银行',	'工商银行',	'建设银行',	'中国建设银行',	'工商银行',	'浙江泰隆商业银行',	'上海农商银行贷记卡',	'建设银行',	'建设银行',	'浙江民泰商业银行',	'中国银行澳门分行',	'中国银行澳门分行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行澳门分行',	'中国银行澳门分行',	'中国银行澳门分行',	'澳门永亨银行股份有限公司',	'曲靖市商业银行',	'宁波银行',	'澳门永亨银行股份有限公司',	'中国银行',	'中行新加坡分行',	'中行新加坡分行',	'永亨银行',	'中国光大银行',	'福建省农村信用社联合社',	'工商银行',	'成都农村商业银行股份有限公司',	'哈尔滨商行',	'中国邮政储蓄银行信用卡中心',	'中国邮政储蓄银行信用卡中心',	'中国邮政储蓄银行信用卡中心',	'成都市商业银行',	'江苏省农村信用社联合社',	'江苏省农村信用社联合社',	'青海银行',	'成都市商业银行',	'中国农业银行贷记卡',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'天津农村商业银行',	'中国农业银行贷记卡',	'农业银行',	'农业银行',	'中国农业银行贷记卡',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'农业银行',	'北京银行',	'北京银行',	'北京银行',	'苏州银行股份有限公司',	'桂林市商业银行',	'日照市商业银行',	'浙江省农村信用社联合社',	'珠海农村商业银行',	'龙江银行股份有限公司',	'澳门永亨银行股份有限公司',	'澳门永亨银行股份有限公司',	'莱芜银行',	'吉林银行',	'徐州市商业银行',	'重庆农村商业银行股份有限公司',	'温州银行',	'太仓农村商业银行',	'江苏长江商业银行',	'永亨银行',	'江苏银行',	'江苏银行',	'徽商银行',	'杭州商业银行',	'徽商银行',	'柳州银行股份有限公司',	'柳州银行股份有限公司',	'尧都区农村信用合作社联社',	'渤海银行',	'重庆农村商业银行股份有限公司',	'烟台商业银行',	'东莞商行',	'中国工商银行(亚洲)有限公司',	'江南农村商业银行',	'上海银行',	'贵州省农村信用社联合社',	'无锡农村商业银行',	'南充市商业银行',	'长沙银行股份有限公司',	'温州银行',	'长沙银行股份有限公司',	'兴业银行',	'工商银行',	'工商银行',	'工商银行',	'湖南农村信用社联合社',	'兴业银行',	'兴业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'日本三井住友卡公司',	'河北银行股份有限公司',	'兴业银行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'中国工商银行澳门分行',	'澳门永亨银行股份有限公司',	'东亚银行中国有限公司',	'吉林农信联合社',	'承德市商业银行',	'德州银行',	'东亚银行中国有限公司',	'遵义市商业银行',	'石嘴山银行',	'AEON信贷财务亚洲有限公司',	'渣打银行中国有限公司',	'东亚银行有限公司',	'中国工商银行(亚洲)有限公司',	'重庆三峡银行',	'汇丰银(中国)有限公司',	'黄河农村商业银行',	'渣打银行香港有限公司',	'中国工商银行(亚洲)有限公司',	'恒生银行',	'恒生银行',	'上海商业银行',	'安徽省农村信用社联合社',	'可汗银行',	'盛京银行',	'永亨银行',	'永亨银行',	'湖北银行',	'邯郸市商业银行',	'安顺市商业银行',	'长治市商业银行',	'永亨银行',	'建设银行',	'赣州银行股份有限公司',	'山西省农村信用社联合社',	'永隆银行有限公司',	'永隆银行有限公司',	'泰安市商业银行',	'乌海银行股份有限公司',	'AEON信贷财务亚洲有限公司',	'内蒙古自治区农村信用社联合式',	'龙江银行股份有限公司',	'鄂尔多斯银行股份有限公司',	'鹤壁银行',	'玉溪市商业银行',	'西安银行股份有限公司',	'张家口市商业银行股份有限公司',	'平安银行（深圳发展银行）（借记卡）',	'上海银行',	'平安银行（深圳发展银行）',	'上海银行',	'建设银行',	'平安银行（深圳发展银行）',	'锦州银行股份有限公司',	'河南省农村信用社联合社',	'广西农村信用社联合社',	'大连银行',	'渣打银行中国有限公司',	'湖北嘉鱼吴江村镇银行',	'成都商业银行',	'成都商业银行',	'中信银行',	'中信银行',	'河北银行股份有限公司',	'广西北部湾银行',	'工商银行',	'秦皇岛银行股份有限公司',	'工商银行',	'临商银行',	'工商银行',	'东莞市商业银行',	'工商银行',	'工商银行',	'河南省农村信用社联合社',	'工商银行',	'工商银行',	'重庆银行',	'青海省农村信用社联合社',	'农业银行',	'厦门银行股份有限公司',	'华夏银行',	'华夏银行',	'华夏银行',	'华夏银行',	'新韩卡公司',	'河北省农村信用社联合社',	'绍兴银行',	'陕西省农村信用社联合社',	'陕西省农村信用社联合社',	'汉口银行',	'朝阳银行',	'东亚银行中国有限公司',	'龙江银行',	'武汉农村商业银行',	'工银新西兰',	'深圳农村商业银行',	'福建省农村信用社联合社',	'漯河银行',	'东莞农村商业银行',	'台州银行',	'中国银行（香港）',	'南洋商业银行',	'集友银行',	'大西洋银行股份有限公司',	'攀枝花市商业银行',	'山西省农村信用社联合社',	'湖北农信社',	'平安银行（深圳发展银行）',	'河南省农村信用社联合社',	'承德银行',	'杭州商业银行',	'中国工商银行',	'福建海峡银行股份有限公司',	'大西洋银行股份有限公司',	'甘肃省农村信用社联合社',	'江苏省农村信用社联合社',	'金华银行股份有限公司',	'衡水银行',	'大连银行',	'大连银行',	'常熟农村商业银行',	'南充市商业银行',	'廊坊银行',	'太仓农村商业银行',	'湖北银行',	'日照银行',	'恒丰银行',	'宜宾市商业银行',	'盛京银行',	'浙江稠州商业银行',	'雅安市商业银行',	'泸州市商业银行',	'绍兴银行',	'枣庄银行',	'成都农村商业银行股份有限公司',	'宁夏银行',	'湖南省农村信用社联合社',	'浙江省农村信用社联合社',	'江西省农村信用社联合社',	'鄂尔多斯银行股份有限公司',	'黄河农村商业银行',	'重庆银行',	'丹东银行',	'抚顺银行',	'工商银行',	'德州银行',	'威海市商业银行',	'天津农村商业银行',	'汉口银行',	'恒生银行',	'恒生银行',	'盛京银行',	'天津滨海农村商业银行',	'吴江农商行',	'北京银行',	'广东华兴银行',	'江阴农村商业银行',	'济宁银行股份有限公司',	'洛阳银行',	'齐商银行',	'石嘴山银行',	'自贡市商业银行',	'张家港农村商业银行',	'无锡农村商业银行',	'广东南粤银行股份有限公司',	'兰州银行股份有限公司',	'吉林银行',	'昆山农信社',	'广西农村信用社联合社',	'玉溪市商业银行',	'驻马店银行',	'昆仑银行股份有限公司',	'辽阳银行股份有限公司',	'平顶山银行股份有限公司',	'乌海银行股份有限公司',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'光大银行',	'信阳银行',	'盘锦市商业银行',	'上海农商银行',	'西安银行股份有限公司',	'阜新银行',	'保定银行',	'青岛银行',	'湖州市商业银行',	'大连银行',	'大连银行',	'营口银行股份有限公司',	'遂宁是商业银行',	'吉林省农村信用社联合社',	'甘肃银行',	'上海银行',	'上海银行',	'深圳农村商业银行',	'鞍山银行',	'江南农村商业银行',	'昆明农联社',	'泰安市商业银行',	'凉山州商业银行',	'宁夏银行',	'四川省农村信用社联合社',	'工商银行',	'徽商银行',	'焦作市商业银行',	'农业银行',	'武汉农村商业银行',	'中国银行',	'杭州商业银行',	'中国建设银行',	'邮政储蓄银行',	'邮政储蓄银行',	'建设银行',	'Rawbank S.a.r.l',	'工商银行',	'PVB Card Corporation',	'PVB Card Corporation',	'PVB Card Corporation',	'U Microfinance Bank Limited',	'南洋商业银行',	'中国银行（香港）',	'集友银行',	'Ecobank Nigeria',	'Al Baraka Bank(Pakistan)',	'OJSC Hamkor bank',	'Fidelity Bank Plc',	'东亚银行有限公司',	'NongHyup Bank',	'PVB Card Corporation',	'中信嘉华银行有限公司',	'State Bank of Mauritius',	'K & R International Limited',	'中国工商银行(亚洲)有限公司',	'JSC ATFBank',	'JSC ATFBank',	'JSC ATFBank',	'OJSC Russian Investment Bank',	'NongHyup Bank',	'JSC Liberty Bank',	'State Bank of Mauritius',	'工商银行',	'工商银行',	'广发银行股份有限公司',	'海口联合农村商业银行',	'黑龙江省农村信用社联合社',	'龙江银行',	'苏州银行股份有限公司',	'邯郸市商业银行',	'工商银行',	'广东华兴银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'工商银行',	'中信嘉华银行有限公司',	'Cim Finance Ltd',	'NongHyup Bank',	'NongHyup Bank',	'Cim Finance Ltd',	'JSC ATFBank',	'NongHyup Bank',	'NongHyup Bank',	'工商银行',	'台州银行',	'越南西贡商业银行',	'BC卡公司',	'Royal Bank Open Stock Company',	'BC卡公司',	'菲律宾RCBC',	'创兴银行有限公司',	'创兴银行有限公司',	'永亨银行',	'BC卡公司',	'BC卡公司',	'Allied Bank',	'澳门商业银行',	'澳门商业银行',	'中国工商银行（澳门）',	'中国工商银行（澳门）',	'中国工商银行（澳门）',	'BC卡公司',	'中国工商银行法兰克福分行',	'中国工商银行法兰克福分行',	'BC卡公司',	'恒生银行有限公司',	'BC卡公司',	'恒生银行有限公司',	'BC卡公司',	'交通银行香港分行',	'交通银行香港分行',	'BC卡公司',	'BC卡公司',	'菲律宾BDO',	'香港上海汇丰银行有限公司',	'菲律宾BDO',	'浙江民泰商业银行',	'BC卡公司',	'中银信用卡(国际)有限公司',	'中银信用卡(国际)有限公司',	'南洋商业银行',	'南洋商业银行',	'集友银行',	'广州银行股份有限公司',	'集友银行',	'中银信用卡(国际)有限公司',	'南洋商业银行',	'集友银行',	'永隆银行有限公司',	'永隆银行有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'花旗中国',	'花旗中国',	'花旗中国',	'花旗中国',	'BC卡公司',	'BC卡公司',	'广州农村商业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'佛山顺德农村商业银行',	'龙江银行股份有限公司',	'花旗银行有限公司',	'恒生银行有限公司',	'南洋商业银行',	'南洋商业银行',	'香港上海汇丰银行有限公司',	'香港上海汇丰银行有限公司',	'上海银行',	'常熟市农村商业银行',	'BC卡公司',	'俄罗斯ORIENT EXPRESS BANK',	'BC卡公司',	'上海商业银行',	'工银新西兰',	'BC卡公司',	'BC卡公司',	'工行米兰',	'中国工商银行巴黎分行',	'中国工商银行(亚洲)有限公司',	'中国工商银行澳门分行',	'哈尔滨银行',	'BC卡公司',	'BC卡公司',	'越南Vietcombank',	'BC卡公司',	'BC卡公司',	'福建省农村信用社联合社',	'BC卡公司',	'BC卡公司',	'乌鲁木齐市商业银行',	'中国银行（香港）',	'BC卡公司',	'中国银行曼谷分行',	'中银信用卡(国际)有限公司',	'中银信用卡(国际)有限公司',	'中国银行澳门分行',	'澳门大丰银行',	'越南Vietcombank',	'临汾市尧都区农村信用合作联社',	'BC卡公司',	'BC卡公司',	'北京农商行',	'大丰银行有限公司',	'大丰银行有限公司',	'BC卡公司',	'BC卡公司',	'BC卡公司',	'东莞农村商业银行',	'BC卡公司',	'中国工商银行',	'中国工商银行',	'中国工商银行',	'中国银行',	'中国农业银行贷记卡',	'中国银行',	'中国银行',	'中国光大银行',	'上海银行',	'上海银行',	'上海银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国建设银行',	'中国建设银行',	'上海农商银行贷记卡',	'乌鲁木齐市商业银行',	'乌鲁木齐市商业银行',	'湖南农信',	'成都农村商业银行',	'湖南农信',	'北京农商行',	'宁夏银行',	'中国银行',	'哈尔滨银行',	'龙江银行',	'河北银行股份有限公司',	'徽商银行',	'徽商银行',	'中国工商银行',	'中国工商银行',	'杭州市商业银行',	'招商银行信用卡中心',	'招商银行信用卡中心',	'韩国KB',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'广发银行股份有限公司',	'韩国三星卡公司',	'韩国三星卡公司',	'天津农村商业银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'中国农业银行贷记卡',	'中国农业银行贷记卡',	'徽商银行',	'Bank of China(Malaysia)',	'浦发银行信用卡中心',	'广州银行股份有限公司',	'上海银行',	'新韩卡公司',	'新韩卡公司',	'Kasikorn Bank PCL',	'中国工商银行',	'中国工商银行',	'东莞农村商业银行',	'中国工商银行',	'中国工商银行(亚洲)有限公司',	'长沙银行股份有限公司',	'江苏银行',	'宁波银行',	'韩国乐天',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'中国银行',	'民生银行',	'民生银行',	'民生银行',	'中国工商银行阿拉木图子行',	'中国工商银行(亚洲)有限公司',	'中国工商银行(亚洲)有限公司',	'中国工商银行新加坡分行',	'中国工商银行印尼分行',	'中国邮政储蓄银行信用卡中心',	'工银泰国',	'工行马来西亚',	'工行河内',	'工行伦敦',	'中国工商银行卢森堡分行',	'工行阿姆斯特丹',	'中国工商银行（巴西）',	'工行阿拉木图',	'工行布鲁塞尔',	'工行加拿大',	'工银印尼',	'工银法兰克福',	'工行马德里',	'工银河内',	'中国工商银行金边分行',	'大新银行有限公司',	'中国工商银行卡拉奇分行',	'Bank of China(Malaysia)',	'汉口银行',	'包商银行',	'哈尔滨银行',	'上海银行',	'建设银行',	'建设银行',	'浦东发展银行',	'浦东发展银行',	'青海银行',	'兴业银行',	'兴业银行',	'兴业银行',	'兴业银行',	'中国建设银行',	'中国建设银行',	'中国建设银行',	'华夏银行',	'华夏银行',	'华夏银行',	'浦发银行信用卡中心',	'浦发银行信用卡中心',	'东亚银行(中国)有限公司',	'东亚银行(中国)有限公司',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国光大银行',	'中国工商银行万象分行',	'中国工商银行金边分行',	'温州银行',	'上海农商银行贷记卡',	'浦东发展银行',	'天津市商业银行',	'农业银行',	'农业银行',	'农业银行',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'汉口银行',	'中国光大银行',	'中国光大银行',	'北京银行',	'金华银行股份有限公司',	'天津银行',	'中信银行信用卡中心',	'宁波银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'江苏银行',	'上海农商银行贷记卡',	'兴业银行',	'贵阳银行股份有限公司',	'宁夏银行',	'交通银行',	'东营银行',	'交通银行',	'桂林银行',	'重庆银行股份有限公司',	'浦东发展银行',	'浦东发展银行',	'上饶银行',	'哈尔滨商行',	'江苏省农村信用社联合社',	'广西北部湾银行',	'成都市商业银行',	'承德银行',	'上海银行',	'富滇银行',	'福建省农村信用社联合社',	'赣州银行股份有限公司',	'威海市商业银行',	'广州农村商业银行',	'杭州市商业银行',	'广东南粤银行',	'九江银行股份有限公司',	'青岛银行',	'南京银行',	'吴江农商行',	'浙江稠州商业银行',	'徽商银行',	'内蒙古银行',	'鄂尔多斯银行',	'温州银行',	'日照银行',	'民生银行',	'广发银行股份有限公司',	'广发银行股份有限公司',	'锦州银行',	'招商银行信用卡中心',	'兰州银行',	'中国建设银行',	'重庆农村商业银行',	'农业银行',	'中国农业银行贷记卡',	'珠海华润银行',	'商丘市商业银行',	'常熟市农村商业银行',	'台州银行',	'浙江泰隆商业银行',	'乌鲁木齐市商业银行',	'莱商银行',	'浙江省农村信用社联合社',	'长沙银行股份有限公司',	'长沙银行股份有限公司',	'长沙银行股份有限公司',	'盛京银行',	'工商银行',	'青海银行',	'工商银行',	'青海省农村信用社联合社',	'招商银行',	'绍兴银行',	'柳州银行',	'包商银行',	'平安银行（深圳发展银行）股份有限公司',	'四川省农村信用社联合社',	'大连银行',	'宁波鄞州农村合作银行',	'天津农村商业银行',	'漯河银行',	'南昌银行',	'湖州银行',	'云南省农村信用社',	'成都农村商业银行股份有限公司',	'辽阳银行股份有限公司',	'中国邮政储蓄银行信用卡中心',	'齐商银行',	'中国银行',	'中国银行',	'周口银行',	'中国建设银行',	'中国建设银行',	'华夏银行',	'泉州银行',	'河北银行股份有限公司',	'佛山顺德农村商业银行',	'陕西省农村信用社联合社',	'华融湘江银行',	'丹东银行',	'广西壮族自治区农村信用社联合社',	'信阳银行',	'甘肃省农村信用社联合社',	'龙江银行股份有限公司',	'北京农商行',	'平顶山银行',	'玉溪市商业银行',	'石嘴山银行',	'吉林银行',	'临商银行',	'招商银行信用卡中心',	'中国建设银行',	'广州银行股份有限公司',	'龙江银行股份有限公司',	'无锡农村商业银行',	'中信银行信用卡中心',	'中信银行信用卡中心',	'中信银行信用卡中心',	'鄂尔多斯银行股份有限公司',	'齐鲁银行股份有限公司',	'江阴农村商业银行',	'临汾市尧都区农村信用合作联社',	'湖南农村信用社联合社',	'中国银行',	'驻马店银行',	'潍坊银行',	'江西省农村信用社联合社',	'德州银行',	'东莞商行',	'泉州银行',	'广发银行股份有限公司',	'招商银行',	'长沙银行股份有限公司',	'工商银行',	'工商银行',	'山东农村信用联合社',	'山东农村信用联合社',	'广州农村商业银行股份有限公司',	'中银通商务支付有限公司',	'中银通支付',	'中银通商务支付有限公司',	'中银通商务支付有限公司',	'乌鲁木齐市商业银行',	'鞍山银行',	'锦州银行股份有限公司',	'南昌银行',	'齐鲁银行股份有限公司',	'吉林银行',	'上海农商银行',	'天津市商业银行',	'广州银行股份有限公司',	'南通商业银行',	'重庆银行',	'重庆农村商业银行股份有限公司',	'上海银行',	'宁波银行',	'厦门银行股份有限公司',	'镇江市商业银行',	'成都商业银行',	'昆山农信社',	'晋商银行',	'焦作市商业银行',	'恒丰银行',	'广州农村商业银行股份有限公司',	'广东南海农村商业银行',	'佛山顺德农村商业银行',	'盛京银行',	'兰州银行股份有限公司',	'洛阳银行',	'昆明农联社',	'辽阳银行股份有限公司',	'武汉农信',	'河北银行股份有限公司',	'吉林银行',	'珠海华润银行股份有限公司',	'哈尔滨银行',	'东莞市商业银行',	'金华银行股份有限公司',	'抚顺银行股份有限公司',	'葫芦岛市商业银行',	'宜昌市商业银行',	'郑州银行股份有限公司',	'齐商银行',	'江阴农村商业银行',	'丹东银行股份有限公司',	'青岛银行',	'东莞农村商业银行',	'张家港农村商业银行',	'日照市商业银行',	'临沂商业银行',	'青海银行',	'台州银行',	'盐城商行',	'长沙银行股份有限公司',	'潍坊银行',	'赣州银行股份有限公司',	'泉州银行',	'江苏银行',	'邮储银行',	'交通银行',	'交通银行',	'交通银行',	'交通银行',	'兴业银行',	'中信银行',	'中信银行',	'中信银行',	'浦东发展银行',	'浦东发展银行',	'佛山顺德农村商业银行',	'汉口银行',	'昆明商业银行',	'平安银行（深圳发展银行）（借记卡）',	'平安银行（深圳发展银行）股份有限公司',	'平安银行（深圳发展银行）股份有限公司',	'华夏银行']
    df_yinhang = pd.DataFrame(card_yinhang)
    dict_yinhang = df_yinhang.set_index('卡标识')['银行'].to_dict()

    #获取本方银行为空的数据
    df3 = df2[df2['本方开户银行'].isna()].copy()
    df3['前6位'] = df3['交易卡号'].str[:6]
    df3.loc[:,'前6位'].replace(dict_yinhang,inplace=True)
    #取出成功替换的行
    df3 = df3[df3['前6位'].isin(list_yinhang)]
    df3['本方开户银行'] = df3['前6位']
    #删除多余列
    df3.drop(['前6位'],axis=1,inplace=True)
    
    df2 = df2.append(df3,sort=False)
    df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    
    #获取对手开户银行是空的数据
    df4 = df2[df2['对手开户银行'].isna()].copy()
    df4['前6位'] = df4['对手卡号'].str[:6]
    df4.loc[:,'前6位'].replace(dict_yinhang,inplace=True)
    #取出成功替换的行
    df4 = df4[df4['前6位'].isin(list_yinhang)]
    df4.loc[:,'对手开户银行'] = df4.loc[:,'前6位']
    df4.drop(['前6位'],axis=1,inplace=True)
    
    df2 = df2.append(df4,sort=False)
    df2.drop_duplicates(subset=['编号'],keep='last',inplace=True)
    return df2
    
def complement_cardB(df2):
    df2.loc[:,'对手卡号'] = df2.loc[:,'对手卡号'].fillna(df2.loc[:,'对手户名'])
    df2.loc[:,'摘要'] = df2.loc[:,'摘要'].fillna('空')
    df2.loc[:,'摘要'] = df2['摘要'].apply(str)
    df2.loc[:,'对手卡号'] = df2.loc[:,'对手卡号'].fillna(df2.loc[:,'交易卡号']+'用途:'+df2.loc[:,'摘要'])
    df2.loc[:,'对手户名'] = df2.loc[:,'对手户名'].fillna(df2.loc[:,'对手卡号'])
    return df2
# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------
if __name__=='__main__':
    #修改当前工作目录
    Folder_Path = r'C:\Users\Liang\Desktop\333'
    os.chdir(Folder_Path)
    
    print("请问有开户信息表吗？‘有’请输入1，‘没有’请输入2")
    one_two = safe_1or2_input('请输入>>： ')
    print()
    
    print("请问要做15位卡号替换成17位卡号吗？（公司账户很多的情况下清洗较慢，可以不做）")
    print("‘不做’请输入1，‘做’请输入2")
    one_two_1517 = safe_1or2_input('请输入>>： ')
    print()
    
    if one_two == 1:
        #加载开户信息
        f = open('开户信息.csv',encoding = 'GB18030')
        df1 = pd.read_csv(f,dtype={'交易卡号':str,'交易账号':str,'客户证件号':str})
        print('清洗开户信息...')
        print()
        #开户信息清洗
        df1 = delete_dash(df1)#清洗：交易卡号、交易账号去除 “_” 下划线后的值
        df1 = fill_value(df1)#清洗：交易卡号为空就填充交易账号的值，交易账号为空就填充交易卡号的值
        df1 = remove_duplicate_kaihu(df1)#开户信息去除重复记录
        
        df1 = df1[~ ((df1['交易户名'].isnull()) & (df1['交易卡号'] == df1['交易账号']))]
        
        df1.to_excel('智器云-开户信息-清洗完成.xlsx',index=0,encoding = 'GB18030')
        print('开户信息清洗完毕！')
        print('生成结果文件《智器云-开户信息-清洗完成.xlsx》')
        print()
        
    print('清洗资金流...')
    print()
    print('加载数据...')
    print()
    #加载资金流数据
    f = open('资金流.csv',encoding = 'GB18030')
    df = pd.read_csv(f,dtype={'交易卡号':str,'交易账号':str,'交易户名':str,'对手卡号':str,'对手账号':str,\
       '对手户名':str,'摘要':str,'备注':str,'IP地址':str,'MAC地址':str,\
           '交易流水号':str,'对手开户银行':str,'交易渠道':str,'交易网点':str,\
            '交易地点':str,'币种':str,'客户证件号':str,'对方证件号':str,'交易流水号':str,\
                '交易是否成功':str,'本方开户银行':str,'对手开户银行':str})
    #日期列转化成日期格式
    df['日期时间'] = pd.to_datetime(df['日期时间'])
    start =time.perf_counter()
    
    print('加载完成，数据有{0}行。'.format(len(df)))
    print()       
    #删除重复行
    df.drop_duplicates(subset=None,keep='first',inplace=True) 
    df.dropna(how = 'all',axis = 0,inplace=True)
    print('去重完全一样的行，数据还剩{0}行。'.format(len(df)))
    print()
    #保留至少有3个非空的行
    df.dropna(thresh=3,inplace=True)
    #删除交易金额为空的行
    df.dropna(subset=['交易金额'],inplace=True)
    #删除交易金额为0的行
    df = df[df['交易金额'] != 0 ]
    #增加‘编号’列，并将索引值赋值过去
    df['编号'] = df.index
    
      
    df = delete_dash(df)#清洗：交易卡号、交易账号去除 “_” 下划线后的值
    
    #'交易卡号'填充
    if one_two == 1:
        df = splicing_table(df1,df,'交易卡号','交易账号')#清洗：依据开户信息的卡号-账号对应关系，填充相应的卡号
    df = splicing_alreadytable(df,'交易卡号','交易账号')#清洗：交易卡号为空根据已有的卡号-账号对应进行填充
    df = fill_value(df)#清洗：交易卡号为空就填充交易账号的值，交易账号为空就填充交易卡号的值
    print('交易卡号填充完成！')
    print()
    
    #'交易户名'填充
    if one_two == 1:
        df = splicing_table(df1,df,'交易户名','交易卡号')#清洗：户名为空则依据交易卡号匹配开户信息中的户名
    df = splicing_alreadytable(df,'交易户名','交易卡号')#清洗：交易户名为空根据已有的卡号-户名对应进行填充
    df = splicing_alreadytableB(df,'交易户名','交易卡号','对手户名','对手卡号')#清洗：交易户名为空根据已有的对手卡号-对手户名对应进行填充
    print('交易户名填充完成！')
    print()
    
    #'对手卡号'填充
    df[['对手卡号','对手账号']]=df[['对手卡号','对手账号']].fillna(axis=1,method='backfill')#清洗：对手卡号为空就填充对手账号的值
    print('对手卡号填充完成！')
    print()
    
    #‘对手户名’填充
    df = splicing_alreadytableB(df,'对手户名','对手卡号','交易户名','交易卡号')#清洗：对手户名为空根据已有的卡号-户名对应进行填充
    df = splicing_alreadytable(df,'对手户名','对手卡号')#清洗：对手户名为空根据已有的对手卡号-对手户名对应进行填充
    print('对手户名填充完成！')
    print()
    
    #‘本方开户银行’‘对手开户银行’填充
    if one_two == 1:
        df = splicing_table(df1,df,'本方开户银行','交易卡号')#清洗：本方开户银行为空则依据交易卡号匹配开户信息中的本方开户银行 
    df = fill_bank(df)# 清洗：依据基础库的卡与银行对应，填充银行
    df = splicing_alreadytableB(df,'本方开户银行','交易卡号','对手开户银行','对手卡号')#本方开户银行为空则依据对手卡号-对手开户银行的对应填充
    print('本方、对手开户银行填充完成！')
    print()
    
    #‘客户证件号’填充
    if one_two == 1:
        df = splicing_table(df1,df,'客户证件号','交易卡号')#清洗：客户证件号为空则依据交易卡号匹配开户信息中的客户证件号
        print('客户证件号填充完成！')
        print()
    
    #清洗：删除对手卡号、对手账号等于“0”
    df = remove_cardB_0(df)       
    #卡号跟账号的值统一
    df = replace_card(df,'交易卡号','交易账号')
    print('卡号与账号统一完成！')
    print()
    
    #清洗交易金额为负的情况
    df = reversal(df)#清洗：交易金额为负，借贷标志为出，则将金额取正，借贷标志设为进（冲正、消费退货、当日冲账）
    df = reversal1(df)#清洗：交易金额为负，借贷标志为进，则将金额取正，借贷标志设为出
    print('交易金额为负已处理完成！')
    print()
    
    #清洗：删除摘要包含“结息”“利息”记录
    # df = delete_interest(df)
    # print('摘要包含结息、利息的记录已删除！')
    # print()
    
    #删除对手卡号中微信、支付宝、财付通等公共账号
    df = delete_public_number(df)
    print('支付宝、财付通等公共账号已删除！')
    print()
    
    if one_two_1517 == 2:
        #如果‘对手卡号’15位的卡号与17位的卡号后15位完全相同，那么将15位的卡号替换成17位的卡号
        df = replace_card1517(df)
        print('15位卡号已替换成17位卡号！')
        print()
    
    #单方去重
    df.drop_duplicates(['交易卡号','日期时间','借贷方向','交易金额','交易余额','对手卡号'],keep='last',inplace=True)
    print('单方去重完成，数据还剩{0}行。'.format(len(df)))
    print()
    
    # 分别指定升序和降序
    df = df.sort_values(by =['交易卡号','日期时间','交易余额'],ascending= [True,True,False])
    #重置索引
    df.reset_index(drop= True,inplace=True)
    df.drop(['编号'], axis=1,inplace=True)
    
    input_file(df,'资金流v2-清洗完成.csv')
    
    
    # 去重2：开始对交易双方的重复项进行去重（根据卡号）
    print('开始对交易双方进行去重...')
    print()
    
    df = remove_duplicate(df,180)
    
    print('交易双方去重完成！')
    input_file(df,'资金流v3-去重完成.csv')
    
     
    #补全对手卡号
    df = complement_cardB(df)#对手卡号为空先填充对手户名,还为空再填充“本方卡号&"用途："&摘要”
    print('补全对手卡号完成！（空值用 本方卡号&"用途："&摘要 填充）')
    print()
    input_file(df,'资金流v4-补全对手.csv')
    end = time.perf_counter()
    print('耗时：%.4f Seconds'%(end-start))

